This notebook allows collecting results across multiple pipeline runs. Concatenating across sessions and bin sizes. 
It takes CSVs, then determines the most recent one from the filename. 

There are two major sets of decoded epochs - Laps and Ripples/Replays
There are two sets of marginals for the decoded epochs - the "by epoch" and "by time bin" marginals.
The "by time bin" epochs are a larger granulation, with each epoch consisting of one or more time bin.


```python

# 2024-01-23 - 
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-fet11-01_12-58-54_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-11-03_12-3-25_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-11-02_17-46-44_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-11-02_19-28-0_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-10_12-58-3_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-10_12-25-50_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-09_16-40-54_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-09_22-24-40_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-12_16-53-46_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-09_17-29-30_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-08_14-26-15_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-09_1-22-43_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-07_16-40-19_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-08_21-16-25_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-12_15-55-31_time_bin_size_sweep_results.h5



# found_session_export_paths = [Path(v).resolve() for v in  ["C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0420PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0420PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0645PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0645PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# ]]

[
    "2024-02-19_0430PM-kdiba_vvp01_one_2006-4-10_12-25-50-(ripple_simple_pf_pearson_merged_df)_tbin-0.025.csv",
    "2024-02-17_0210AM-kdiba_vvp01_one_2006-4-09_17-29-30-(laps_simple_pf_pearson_merged_df)_tbin-0.025.csv",
    "2024-02-19_0430PM-kdiba_vvp01_one_2006-4-10_12-25-50-(laps_weighted_corr_merged_df)_tbin-0.025.csv",
    "2024-02-19_0430PM-kdiba_vvp01_one_2006-4-10_12-25-50-(ripple_weighted_corr_merged_df)_tbin-0.025.csv",
]

```

In [3]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3

# required to enable non-blocking interaction:
%gui qt5

import sys
from copy import deepcopy
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from typing_extensions import TypeAlias
from neuropy.utils.result_context import IdentifyingContext
from nptyping import NDArray
import neuropy.utils.type_aliases as types
from collections import defaultdict

import numpy as np
import pandas as pd
import re
from pathlib import Path
from datetime import datetime

from neuropy.utils.indexing_helpers import PandasHelpers
from pyphocorehelpers.indexing_helpers import partition_df
# Set the maximum number of columns to display
pd.set_option('display.max_columns', 100)

import IPython
from pyphocorehelpers.programming_helpers import IPythonHelpers
from pyphocorehelpers.notebook_helpers import NotebookCellExecutionLogger

_notebook_path:Path = Path(IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())).resolve() # Finds the path of THIS notebook
_notebook_execution_logger: NotebookCellExecutionLogger = NotebookCellExecutionLogger(notebook_path=_notebook_path, enable_logging_to_file=True) # Builds a logger that records info about this notebook

# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()

matplotlib.use('Qt5Agg')
# %matplotlib inline
# %matplotlib auto

# Switch to the desired interactivity mode
plt.interactive(True)

# _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
template: str = 'plotly_dark' # set plotl template
pio.templates.default = template
from pyphocorehelpers.plotting.media_output_helpers import fig_to_clipboard
from pyphocorehelpers.Filesystem.path_helpers import file_uri_from_path, sanitize_filename_for_Windows
from pyphocorehelpers.gui.Jupyter.simple_widgets import fullwidth_path_widget, simple_path_display_widget
from pyphoplacecellanalysis.Pho2D.plotly.Extensions.plotly_helpers import plotly_helper_save_figures, _helper_build_figure, plotly_pre_post_delta_scatter, plot_across_sessions_scatter_results

# from ..PendingNotebookCode import plot_across_sessions_scatter_results, plot_histograms, plot_stacked_histograms
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import find_csv_files, find_HDF5_files, find_most_recent_files, process_csv_file, process_parsed_csv_files_df
from pyphoplacecellanalysis.Pho2D.statistics_plotting_helpers import plot_histograms_across_sessions, plot_histograms, plot_stacked_histograms

from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DecoderDecodedEpochsResult
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import load_across_sessions_exported_files, _process_and_load_exported_file, _common_cleanup_operations, convert_to_dataframe

from pyphocorehelpers.programming_helpers import metadata_attributes
from pyphocorehelpers.function_helpers import function_attributes

from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import recover_user_annotation_and_is_valid_columns

debug_print: bool = False
enable_neptune: bool = False

TODAY_DAY_DATE: str = f"2024-07-09_Apogee"
# TODAY_DAY_DATE: str = f"2024-07-09_GL"
# TODAY_DAY_DATE: str = f"2024-07-09_rMBP"

print(f'TODAY_DAY_DATE: {TODAY_DAY_DATE}')

types.session_str: TypeAlias = str # a unique session identifier

if enable_neptune:
    import neptune # for logging progress and results
    from neptune.types import File
    from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import Neptuner, AutoValueConvertingNeptuneRun, set_environment_variables 

    ## Gets the notebook filepath for Neptune:
    import IPython
    from pyphocorehelpers.programming_helpers import IPythonHelpers
    notebook_filepath: str = IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())
    assert Path(notebook_filepath).resolve().exists(), f"found notebook filepath: '{notebook_filepath}' does not exist"
    # notebook_filepath

    neptune_kwargs = {'project':"commander.pho/PhoDibaLongShortAcrossSessions",
    'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}
            
    neptuner = Neptuner(project_name=neptune_kwargs['project'], api_token=neptune_kwargs['api_token'])


    if neptuner.run is None:
        neptuner.run = AutoValueConvertingNeptuneRun(project=neptuner.project_name, api_token=neptuner.api_token, dependencies="infer", source_files=[notebook_filepath])
        params = {"TODAY_DAY_DATE": TODAY_DAY_DATE, "run_workstation": "Apogee"}
        neptuner.run["parameters"] = params
        neptuner.outputs = neptuner.run['outputs']
        neptuner.figures = neptuner.outputs['figures']

    neptuner_run: AutoValueConvertingNeptuneRun = neptuner.run
    
    # run = neptune.init_run(source_files=["**/*.dvc"])

    # # Pre-execution dataframe view:
    # run["dataset/global_batch_run_progress_df"].upload(File.as_html(global_batch_run.to_dataframe(expand_context=True, good_only=False))) # "path/to/test_preds.csv"

else:
    # no neptune:
    neptuner = None    
    neptuner_run = None


known_bad_sessions = [IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44')]
known_bad_session_strs = [str(v.get_description()) for v in known_bad_sessions]
known_bad_session_strs

Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
TODAY_DAY_DATE: 2024-07-09_Apogee


['kdiba_pin01_one_11-02_17-46-44']

Cell executed in: 17 ms
Cell executed in: 17 ms


# Definitions
"_new" (as in `all_sessions_new_ripple_df`) adds simple_pearson and a few other measures

### csv_files

In [4]:
## Load across session t_delta CSV, which contains the t_delta for each session:

## INPUTS: known_bad_session_strs,

cuttoff_date = datetime(2024, 7, 5)
# cuttoff_date = datetime(2024, 5, 18)
# cuttoff_date = None

# t_delta_csv_path = Path(r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\output\collected_outputs\2024-01-18_GL_t_split_df.csv').resolve() # Apogee
# t_delta_csv_path = Path('/home/halechr/cloud/turbo/Data/Output/collected_outputs/2024-01-18_GL_t_split_df.csv').resolve() # GL

# collected_outputs_directory = '/home/halechr/FastData/collected_outputs/'
# collected_outputs_directory = r'C:\Users\pho\Desktop\collected_outputs'
# collected_outputs_directory = r'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs' # APOGEE
# collected_outputs_directory = '/home/halechr/cloud/turbo/Data/Output/collected_outputs' # GL

known_collected_outputs_paths = [Path(v).resolve() for v in ['/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs',
                                                            '/Volumes/SwapSSD/Data/collected_outputs', r"K:/scratch/collected_outputs", '/Users/pho/Dropbox (University of Michigan)/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs', r'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs',
                                                            '/home/halechr/FastData/collected_outputs/', '/home/halechr/cloud/turbo/Data/Output/collected_outputs']]
collected_outputs_directory = find_first_extant_path(known_collected_outputs_paths)
assert collected_outputs_directory.exists(), f"collected_outputs_directory: {collected_outputs_directory} does not exist! Is the right computer's config commented out above?"
# fullwidth_path_widget(scripts_output_path, file_name_label='Scripts Output Path:')
print(f'collected_outputs_directory: {collected_outputs_directory}')

# Create a 'figures' subfolder if it doesn't exist
figures_folder: Path = collected_outputs_directory.joinpath('figures').resolve()
figures_folder.mkdir(parents=False, exist_ok=True)
assert figures_folder.exists()
print(f'\tfigures_folder: {file_uri_from_path(figures_folder)}')

# Create an output path for the across session collected results (like the aggregate CSVs built from the individual session CSVs)
across_sessions_output_folder: Path = collected_outputs_directory.joinpath('../across_sessions').resolve()
across_sessions_output_folder.mkdir(parents=False, exist_ok=True)
assert across_sessions_output_folder.exists()
print(f'\tacross_sessions_output_folder: {file_uri_from_path(across_sessions_output_folder)}')

# t_delta_csv_path = collected_outputs_directory.joinpath('2024-01-18_GL_t_split_df.csv').resolve() # GL
t_delta_csv_path = collected_outputs_directory.joinpath('../2024-01-18_GL_t_split_df.csv').resolve()
assert t_delta_csv_path.exists(), f"t_split_df CSV at '{t_delta_csv_path}' does not exist!"

if neptuner_run is not None:
    _neptuner_run_parameters = dict(cuttoff_date=cuttoff_date, collected_outputs_directory=collected_outputs_directory.as_posix(), figures_folder=figures_folder.as_posix(),
                           across_sessions_output_folder=across_sessions_output_folder.as_posix(), t_delta_csv_path=t_delta_csv_path.as_posix())
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'parameters/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing

## Find the files:
csv_files = find_csv_files(collected_outputs_directory)
h5_files = find_HDF5_files(collected_outputs_directory)

csv_sessions = find_most_recent_files(found_session_export_paths=csv_files, cuttoff_date=cuttoff_date)
h5_sessions = find_most_recent_files(found_session_export_paths=h5_files)

## INPUTS: csv_sessions
parsed_csv_files_df: pd.DataFrame = convert_to_dataframe(csv_sessions)
parsed_csv_files_df = process_parsed_csv_files_df(parsed_csv_files_df=parsed_csv_files_df)
## OUTPUTS: parsed_csv_files_df

## OUTPUTS: csv_files, csv_sessions

_neptuner_run_parameters = dict(csv_files=csv_files, h5_files=h5_files, csv_sessions=csv_sessions, h5_sessions=h5_sessions)

## The CSV containing the session delta time:
t_delta_df = pd.read_csv(t_delta_csv_path, index_col=0) # Assuming that your CSV file has an index column
# adds `delta_aligned_t_start`, `delta_aligned_t_end` columns
t_delta_df['delta_aligned_t_start'] = t_delta_df['t_start'] - t_delta_df['t_delta']
t_delta_df['delta_aligned_t_end'] = t_delta_df['t_end'] - t_delta_df['t_delta']

# computes `earliest_delta_aligned_t_start`, latest_delta_aligned_t_end
earliest_delta_aligned_t_start: float = np.nanmin(t_delta_df['delta_aligned_t_start'])
latest_delta_aligned_t_end: float = np.nanmax(t_delta_df['delta_aligned_t_end'])
print(f'earliest_delta_aligned_t_start: {earliest_delta_aligned_t_start}, latest_delta_aligned_t_end: {latest_delta_aligned_t_end}')
t_delta_dict = t_delta_df.to_dict(orient='index')
# t_delta_df

# #TODO 2024-03-02 12:12: - [ ] Could add weighted correlation if there is a dataframe for that and it's computed:
_df_raw_variable_names = ['simple_pf_pearson_merged_df', 'weighted_corr_merged_df']
_df_variables_names = ['laps_weighted_corr_merged_df', 'ripple_weighted_corr_merged_df', 'laps_simple_pf_pearson_merged_df', 'ripple_simple_pf_pearson_merged_df']

# # tbin_values_dict = {'laps': self.laps_decoding_time_bin_size, 'ripple': self.ripple_decoding_time_bin_size}
time_col_name_dict = {'laps': 'lap_start_t', 'ripple': 'ripple_start_t'} ## default should be 't_bin_center'

# fold older files:
# {'laps_marginals_df': 'lap_start_t', 'ripple_marginals_df': 'ripple_start_t', 'laps_time_bin_marginals_df':'t_bin_center', 'ripple_time_bin_marginals_df':'t_bin_center'}
    
if neptuner_run is not None:
    _neptuner_run_parameters = _neptuner_run_parameters | dict(earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
                                     t_delta_df=t_delta_df)
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'parsed/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing


collected_outputs_directory: K:\scratch\collected_outputs
	figures_folder: file:///K:/scratch/collected_outputs/figures
	across_sessions_output_folder: file:///K:/scratch/across_sessions
ERR: Could not parse filename: "session_h5_files_copydict_2024-06-06_Lab"
ERR: Could not parse filename: "session_h5_files_copydict_2024-06-11_GL"
ERR: Could not parse filename: "session_h5_files_copydict_2024-06-11_Apogee"
ERR: Could not parse filename: "session_h5_files_copydict_2024-06-12_Apogee"
ERR: Could not parse filename: "session_h5_files_copydict_2024-04-27_Apogee"
earliest_delta_aligned_t_start: -2057.2259484970764, latest_delta_aligned_t_end: 1661.8560019930592
Cell executed in: 152 ms
Cell executed in: 151 ms
Cell executed in: 149 ms


In [5]:

# csv_sessions
# Extract each of the separate files from the sessions:

# final_sessions: Dict[types.session_str, Dict[str, Path]] = {}
final_sessions_loaded_laps_dict = {}
final_sessions_loaded_ripple_dict = {}
final_sessions_loaded_laps_time_bin_dict = {}
final_sessions_loaded_ripple_time_bin_dict = {}

final_sessions_loaded_simple_pearson_laps_dict = {}
final_sessions_loaded_simple_pearson_ripple_dict = {}

# stupid reudndant but compatible method
final_sessions_loaded_laps_wcorr_dict = {}
final_sessions_loaded_ripple_wcorr_dict = {}

final_sessions_loaded_laps_all_scores_dict = {}
final_sessions_loaded_ripple_all_scores_dict = {}

## INPUTS: csv_sessions
if cuttoff_date is not None:
    final_sessions: Dict[types.session_str, Dict[str, Path]] = {session_str:{file_type:a_path for file_type, (a_path, an_decoding_time_bin_size_str, an_export_datetime) in session_dict.items() if ((an_export_datetime >= cuttoff_date) and (session_str not in known_bad_session_strs))}
                                                                                              for session_str, session_dict in csv_sessions.items() }
else:
    # no cutoff recency date:
    final_sessions: Dict[types.session_str, Dict[str, Path]] = {session_str:{file_type:a_path for file_type, (a_path, an_decoding_time_bin_size_str, an_export_datetime) in session_dict.items() if (session_str not in known_bad_session_strs)}
                                                                                            for session_str, session_dict in csv_sessions.items()}

for session_str, session_dict in final_sessions.items():
    session_name = str(session_str)  # Extract session name from the filename
    if debug_print:
        print(f'processing session_name: {session_name}')
    curr_session_t_delta = t_delta_dict.get(session_name, {}).get('t_delta', None)
    if curr_session_t_delta is None:
        print(f'WARN: curr_session_t_delta is None for session_str = "{session_str}"')

    # Process each file type with its corresponding details
    _process_and_load_exported_file(session_dict, 'laps_marginals_df', final_sessions_loaded_laps_dict, session_str, curr_session_t_delta, 'lap_start_t')
    _process_and_load_exported_file(session_dict, 'ripple_marginals_df', final_sessions_loaded_ripple_dict, session_str, curr_session_t_delta, 'ripple_start_t')
    _process_and_load_exported_file(session_dict, 'laps_time_bin_marginals_df', final_sessions_loaded_laps_time_bin_dict, session_str, curr_session_t_delta, 't_bin_center')
    _process_and_load_exported_file(session_dict, 'ripple_time_bin_marginals_df', final_sessions_loaded_ripple_time_bin_dict, session_str, curr_session_t_delta, 't_bin_center')
    _process_and_load_exported_file(session_dict, 'laps_simple_pf_pearson_merged_df', final_sessions_loaded_simple_pearson_laps_dict, session_str, curr_session_t_delta, 'lap_start_t')
    _process_and_load_exported_file(session_dict, 'ripple_simple_pf_pearson_merged_df', final_sessions_loaded_simple_pearson_ripple_dict, session_str, curr_session_t_delta, 'ripple_start_t')
    _process_and_load_exported_file(session_dict, 'laps_weighted_corr_merged_df', final_sessions_loaded_laps_wcorr_dict, session_str, curr_session_t_delta, 'lap_start_t')
    _process_and_load_exported_file(session_dict, 'ripple_weighted_corr_merged_df', final_sessions_loaded_ripple_wcorr_dict, session_str, curr_session_t_delta, 'ripple_start_t')
    
    # _process_and_load_exported_file(session_dict, 'laps_all_scores_merged_df', final_sessions_loaded_laps_all_scores_dict, session_str, curr_session_t_delta, 'lap_start_t')
    _process_and_load_exported_file(session_dict, 'ripple_all_scores_merged_df', final_sessions_loaded_ripple_all_scores_dict, session_str, curr_session_t_delta, 'ripple_start_t')

final_sessions_loaded_ripple_dict


WARN: curr_session_t_delta is None for session_str = "kdiba_gor01_one_2006-6-08_14-26-15__withNewKamranExportedReplays-qclu_[1,2]-frateThresh_5.0"
session "kdiba_gor01_one_2006-6-08_14-26-15__withNewKamranExportedReplays-qclu_[1,2]-frateThresh_5.0", df_file_name_key: "laps_marginals_df" - did not fully work. (error "'laps_marginals_df'". Skipping.
session "kdiba_gor01_one_2006-6-08_14-26-15__withNewKamranExportedReplays-qclu_[1,2]-frateThresh_5.0", df_file_name_key: "ripple_marginals_df" - did not fully work. (error "'ripple_marginals_df'". Skipping.
session "kdiba_gor01_one_2006-6-08_14-26-15__withNewKamranExportedReplays-qclu_[1,2]-frateThresh_5.0", df_file_name_key: "laps_time_bin_marginals_df" - did not fully work. (error "'laps_time_bin_marginals_df'". Skipping.
session "kdiba_gor01_one_2006-6-08_14-26-15__withNewKamranExportedReplays-qclu_[1,2]-frateThresh_5.0", df_file_name_key: "ripple_time_bin_marginals_df" - did not fully work. (error "'ripple_time_bin_marginals_df'". Skippin

{'kdiba_gor01_one_2006-6-08_14-26-15__withNewComputedReplays-qclu_[1, 2]-frateThresh_5.0':      Unnamed: 0      P_LR      P_RL    P_Long   P_Short  ripple_idx  \
 0             0  0.862064  0.137936  0.898964  0.101036           0   
 1             1  0.870587  0.129413  0.474199  0.525801           1   
 2             2  0.720703  0.279297  0.623205  0.376795           2   
 3             3  0.784330  0.215670  0.598649  0.401351           3   
 4             4  0.245822  0.754178  0.655021  0.344979           4   
 ..          ...       ...       ...       ...       ...         ...   
 648         648  0.679935  0.320065  0.563190  0.436810         648   
 649         649  0.762954  0.237046  0.674514  0.325486         649   
 650         650  0.854687  0.145313  0.718408  0.281592         650   
 651         651  0.825422  0.174578  0.384553  0.615447         651   
 652         652  0.542510  0.457490  0.670122  0.329878         652   
 
      ripple_start_t                        

Cell executed in: 192 ms
Cell executed in: 192 ms
Cell executed in: 192 ms


In [6]:

# final_sessions: Dict[types.session_str, Dict[str, Path]] = {}
final_sessions_loaded_laps_dict = {}
final_sessions_loaded_ripple_dict = {}
final_sessions_loaded_laps_time_bin_dict = {}
final_sessions_loaded_ripple_time_bin_dict = {}

final_sessions_loaded_simple_pearson_laps_dict = {}
final_sessions_loaded_simple_pearson_ripple_dict = {}

# stupid reudndant but compatible method
final_sessions_loaded_laps_wcorr_dict = {}
final_sessions_loaded_ripple_wcorr_dict = {}

final_sessions_loaded_laps_all_scores_dict = {}
final_sessions_loaded_ripple_all_scores_dict = {}

## INPUTS: csv_sessions
if cuttoff_date is not None:
    final_sessions: Dict[types.session_str, Dict[str, Path]] = {session_str:{file_type:a_path for file_type, (a_path, an_decoding_time_bin_size_str, an_export_datetime) in session_dict.items() if ((an_export_datetime >= cuttoff_date) and (session_str not in known_bad_session_strs))}
                                                                                              for session_str, session_dict in csv_sessions.items() }
else:
    # no cutoff recency date:
    final_sessions: Dict[types.session_str, Dict[str, Path]] = {session_str:{file_type:a_path for file_type, (a_path, an_decoding_time_bin_size_str, an_export_datetime) in session_dict.items() if (session_str not in known_bad_session_strs)}
                                                                                            for session_str, session_dict in csv_sessions.items()}

for session_str, session_dict in final_sessions.items():
    session_name = str(session_str)  # Extract session name from the filename
    if debug_print:
        print(f'processing session_name: {session_name}')
    curr_session_t_delta = t_delta_dict.get(session_name, {}).get('t_delta', None)
    if curr_session_t_delta is None:
        print(f'WARN: curr_session_t_delta is None for session_str = "{session_str}"')

    # Process each file type with its corresponding details
    _process_and_load_exported_file(session_dict, 'laps_marginals_df', final_sessions_loaded_laps_dict, session_str, curr_session_t_delta, 'lap_start_t')
    _process_and_load_exported_file(session_dict, 'ripple_marginals_df', final_sessions_loaded_ripple_dict, session_str, curr_session_t_delta, 'ripple_start_t')
    _process_and_load_exported_file(session_dict, 'laps_time_bin_marginals_df', final_sessions_loaded_laps_time_bin_dict, session_str, curr_session_t_delta, 't_bin_center')
    _process_and_load_exported_file(session_dict, 'ripple_time_bin_marginals_df', final_sessions_loaded_ripple_time_bin_dict, session_str, curr_session_t_delta, 't_bin_center')
    _process_and_load_exported_file(session_dict, 'laps_simple_pf_pearson_merged_df', final_sessions_loaded_simple_pearson_laps_dict, session_str, curr_session_t_delta, 'lap_start_t')
    _process_and_load_exported_file(session_dict, 'ripple_simple_pf_pearson_merged_df', final_sessions_loaded_simple_pearson_ripple_dict, session_str, curr_session_t_delta, 'ripple_start_t')
    _process_and_load_exported_file(session_dict, 'laps_weighted_corr_merged_df', final_sessions_loaded_laps_wcorr_dict, session_str, curr_session_t_delta, 'lap_start_t')
    _process_and_load_exported_file(session_dict, 'ripple_weighted_corr_merged_df', final_sessions_loaded_ripple_wcorr_dict, session_str, curr_session_t_delta, 'ripple_start_t')
    
    # _process_and_load_exported_file(session_dict, 'laps_all_scores_merged_df', final_sessions_loaded_laps_all_scores_dict, session_str, curr_session_t_delta, 'lap_start_t')
    _process_and_load_exported_file(session_dict, 'ripple_all_scores_merged_df', final_sessions_loaded_ripple_all_scores_dict, session_str, curr_session_t_delta, 'ripple_start_t')

final_sessions_loaded_ripple_dict


WARN: curr_session_t_delta is None for session_str = "kdiba_gor01_one_2006-6-08_14-26-15__withNewKamranExportedReplays-qclu_[1,2]-frateThresh_5.0"
session "kdiba_gor01_one_2006-6-08_14-26-15__withNewKamranExportedReplays-qclu_[1,2]-frateThresh_5.0", df_file_name_key: "laps_marginals_df" - did not fully work. (error "'laps_marginals_df'". Skipping.
session "kdiba_gor01_one_2006-6-08_14-26-15__withNewKamranExportedReplays-qclu_[1,2]-frateThresh_5.0", df_file_name_key: "ripple_marginals_df" - did not fully work. (error "'ripple_marginals_df'". Skipping.
session "kdiba_gor01_one_2006-6-08_14-26-15__withNewKamranExportedReplays-qclu_[1,2]-frateThresh_5.0", df_file_name_key: "laps_time_bin_marginals_df" - did not fully work. (error "'laps_time_bin_marginals_df'". Skipping.
session "kdiba_gor01_one_2006-6-08_14-26-15__withNewKamranExportedReplays-qclu_[1,2]-frateThresh_5.0", df_file_name_key: "ripple_time_bin_marginals_df" - did not fully work. (error "'ripple_time_bin_marginals_df'". Skippin

{'kdiba_gor01_one_2006-6-08_14-26-15__withNewComputedReplays-qclu_[1, 2]-frateThresh_5.0':      Unnamed: 0      P_LR      P_RL    P_Long   P_Short  ripple_idx  \
 0             0  0.862064  0.137936  0.898964  0.101036           0   
 1             1  0.870587  0.129413  0.474199  0.525801           1   
 2             2  0.720703  0.279297  0.623205  0.376795           2   
 3             3  0.784330  0.215670  0.598649  0.401351           3   
 4             4  0.245822  0.754178  0.655021  0.344979           4   
 ..          ...       ...       ...       ...       ...         ...   
 648         648  0.679935  0.320065  0.563190  0.436810         648   
 649         649  0.762954  0.237046  0.674514  0.325486         649   
 650         650  0.854687  0.145313  0.718408  0.281592         650   
 651         651  0.825422  0.174578  0.384553  0.615447         651   
 652         652  0.542510  0.457490  0.670122  0.329878         652   
 
      ripple_start_t                        

Cell executed in: 144 ms
Cell executed in: 144 ms
Cell executed in: 144 ms


In [7]:

## Build across_sessions join dataframes:
all_sessions_laps_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_laps_dict.values()), axis='index', ignore_index=True)
all_sessions_ripple_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_ripple_dict.values()), axis='index', ignore_index=True)
# Add 'epoch_idx' column for compatibility:
if all_sessions_laps_df is not None:
    all_sessions_laps_df['epoch_idx'] = all_sessions_laps_df['lap_idx']
if all_sessions_ripple_df is not None:
    all_sessions_ripple_df['epoch_idx'] = all_sessions_ripple_df['ripple_idx']

# *_time_bin marginals:
all_sessions_laps_time_bin_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_laps_time_bin_dict.values()), axis='index', ignore_index=True)
all_sessions_ripple_time_bin_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_ripple_time_bin_dict.values()), axis='index', ignore_index=True)

# NEW ________________________________________________________________________________________________________________ #
all_sessions_simple_pearson_laps_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_simple_pearson_laps_dict.values()), axis='index', ignore_index=True)
all_sessions_simple_pearson_ripple_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_simple_pearson_ripple_dict.values()), axis='index', ignore_index=True)

if len(final_sessions_loaded_laps_wcorr_dict) > 0:
    all_sessions_wcorr_laps_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_laps_wcorr_dict.values()), axis='index', ignore_index=True)
else:
    all_sessions_wcorr_laps_df = None # empty df would be better

all_sessions_wcorr_ripple_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_ripple_wcorr_dict.values()), axis='index', ignore_index=True)

# `*_all_scores_*`: __________________________________________________________________________________________________ #
# all_sessions_all_score_laps_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_laps_all_scores_dict.values()), axis='index', ignore_index=True)
all_sessions_all_scores_ripple_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_ripple_all_scores_dict.values()), axis='index', ignore_index=True)

# dfs_list = (all_sessions_laps_df, all_sessions_ripple_df, all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df)
dfs_dict = dict(zip(('all_sessions_laps_df', 'all_sessions_ripple_df', 'all_sessions_laps_time_bin_df', 'all_sessions_ripple_time_bin_df'), (all_sessions_laps_df, all_sessions_ripple_df, all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df)))

# for a_df in dfs_list:
for a_df_name, a_df in dfs_dict.items():
    if a_df is not None:
        if 'time_bin_size' not in a_df:
            print(f'Uh-oh! time_bin_size is missing for "{a_df_name}"! This must be old exports!')
            print(f'\tTry to determine the time_bin_size from the filenames: {csv_sessions}')
            ## manual correction UwU
            time_bin_size: float = 0.025
            print(f'\tWARNING! MANUAL OVERRIDE TIME BIN SIZE SET: time_bin_size = {time_bin_size}. Assigning to dataframes....')
            a_df['time_bin_size'] = time_bin_size
        else:
            # Filter rows based on column: 'time_bin_size'
            a_df = a_df[a_df['time_bin_size'].notna()]

all_sessions_laps_df, all_sessions_ripple_df, all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df = [_common_cleanup_operations(a_df) for a_df in (all_sessions_laps_df, all_sessions_ripple_df, all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df)]
all_sessions_simple_pearson_laps_df, all_sessions_simple_pearson_ripple_df, all_sessions_wcorr_laps_df, all_sessions_wcorr_ripple_df = [_common_cleanup_operations(a_df) for a_df in (all_sessions_simple_pearson_laps_df, all_sessions_simple_pearson_ripple_df, all_sessions_wcorr_laps_df, all_sessions_wcorr_ripple_df)]

# all_sessions_all_score_laps_df, all_sessions_all_scores_ripple_df = [_common_cleanup_operations(a_df) for a_df in (all_sessions_all_score_laps_df, all_sessions_all_scores_ripple_df)]
# all_sessions_all_score_laps_df = _common_cleanup_operations(all_sessions_all_score_laps_df)
all_sessions_all_scores_ripple_df = _common_cleanup_operations(all_sessions_all_scores_ripple_df)

all_sessions_simple_pearson_laps_df: pd.DataFrame = DecoderDecodedEpochsResult.merge_decoded_epochs_result_dfs(all_sessions_simple_pearson_laps_df, all_sessions_wcorr_laps_df, should_drop_directional_columns=False, start_t_idx_name='delta_aligned_start_t')
all_sessions_simple_pearson_ripple_df: pd.DataFrame = DecoderDecodedEpochsResult.merge_decoded_epochs_result_dfs(all_sessions_simple_pearson_ripple_df, all_sessions_wcorr_ripple_df, should_drop_directional_columns=False, start_t_idx_name='ripple_start_t')
# all_sessions_laps_time_bin_df # 601845 rows × 9 column

## epoch-based ones:
all_sessions_ripple_df = recover_user_annotation_and_is_valid_columns(all_sessions_ripple_df, all_sessions_all_scores_df=all_sessions_all_scores_ripple_df, a_time_column_names='ripple_start_t')



ERROR: failed with error Must have 'start' column. while trying to add column "is_user_annotated_epoch". Out of options.
ERROR: failed with error Must have 'start' column. while trying to add column "is_valid_epoch". Out of options.
Cell executed in: 199 ms
Cell executed in: 198 ms
Cell executed in: 197 ms


In [ ]:
all_sessions_ripple_time_bin_df

In [ ]:

if neptuner_run is not None:
    _neptuner_run_parameters = dict(parsed_csv_files_df=parsed_csv_files_df,all_sessions_laps_df=all_sessions_laps_df, all_sessions_ripple_df=all_sessions_ripple_df,
                                    all_sessions_laps_time_bin_df=all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df=all_sessions_ripple_time_bin_df,
                                    all_sessions_all_scores_ripple_df=all_sessions_all_scores_ripple_df,
                                    all_sessions_simple_pearson_laps_df=all_sessions_simple_pearson_laps_df, all_sessions_simple_pearson_ripple_df=all_sessions_simple_pearson_ripple_df,
                                       )
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'parsed/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing


# if (neptuner is not None) and (neptuner_run is not None):
#     neptuner.stop()

In [ ]:
# pd.set_option('display.min_rows', 50)
# pd.set_option('display.max_rows', 50)
# pd.set_option('display.show_dimensions', True)

with pd.option_context('display.min_rows', 50, 'display.show_dimensions', True):
    # max_elements
    display(parsed_csv_files_df)

# parsed_csv_files_df[parsed_csv_files_df['file_type'] == 'ripple_all_scores_merged_df']


In [ ]:
parsed_csv_files_df[parsed_csv_files_df['replay_epoch_names'] == 'withNewKamranExportedReplays']

In [ ]:
parsed_csv_files_df[parsed_csv_files_df['file_type'] == 'ripple_all_scores_merged_df']

# all_sessions_all_scores_ripple_df


In [ ]:
all_sessions_ripple_time_bin_df

In [ ]:
print(list(np.unique(parsed_csv_files_df.file_type))) # ['laps_marginals_df', 'laps_simple_pf_pearson_merged_df', 'laps_time_bin_marginals_df', 'laps_weighted_corr_merged_df', 'merged_complete_epoch_stats_df', 'ripple_all_scores_merged_df', 'ripple_marginals_df', 'ripple_simple_pf_pearson_merged_df', 'ripple_time_bin_marginals_df', 'ripple_weighted_corr_merged_df']

In [ ]:
csv_sessions

In [8]:
display(parsed_csv_files_df)

across_sessions_parsed_csv_files_path = across_sessions_output_folder.joinpath(f'{TODAY_DAY_DATE}_parsed_csv_files_df.csv').resolve()
# parsed_csv_files_df.to_clipboard(excel=True)
parsed_csv_files_df.to_csv(across_sessions_parsed_csv_files_path)
display(fullwidth_path_widget(across_sessions_parsed_csv_files_path, file_name_label='across_sessions_parsed_csv_files_path:'))

,session,file_type,path,decoding_time_bin_size_str,export_datetime,replay_epoch_names
0,kdiba_gor01_one_2006-6-08_14-26-15,ripple_weighted_corr_merged_df,K:\scratch\collected_outputs\2024-07-09_0900AM...,0.025,2024-07-09 09:00:00,"withNewKamranExportedReplays-qclu_[1,2]-frateT..."
1,kdiba_gor01_one_2006-6-08_14-26-15,ripple_simple_pf_pearson_merged_df,K:\scratch\collected_outputs\2024-07-09_0900AM...,0.025,2024-07-09 09:00:00,"withNewKamranExportedReplays-qclu_[1,2]-frateT..."
2,kdiba_gor01_one_2006-6-08_14-26-15,ripple_all_scores_merged_df,K:\scratch\collected_outputs\2024-07-09_0900AM...,0.025,2024-07-09 09:00:00,"withNewKamranExportedReplays-qclu_[1,2]-frateT..."
3,kdiba_gor01_one_2006-6-08_14-26-15,laps_weighted_corr_merged_df,K:\scratch\collected_outputs\2024-07-09_0900AM...,0.025,2024-07-09 09:00:00,"withNewKamranExportedReplays-qclu_[1,2]-frateT..."
4,kdiba_gor01_one_2006-6-08_14-26-15,laps_simple_pf_pearson_merged_df,K:\scratch\collected_outputs\2024-07-09_0900AM...,0.025,2024-07-09 09:00:00,"withNewKamranExportedReplays-qclu_[1,2]-frateT..."
5,kdiba_gor01_one_2006-6-08_14-26-15,ripple_time_bin_marginals_df,K:\scratch\collected_outputs\2024-07-09-kdiba_...,None,2024-07-09 00:00:00,"withNewComputedReplays-qclu_[1, 2]-frateThresh..."
6,kdiba_gor01_one_2006-6-08_14-26-15,ripple_marginals_df,K:\scratch\collected_outputs\2024-07-09-kdiba_...,None,2024-07-09 00:00:00,"withNewComputedReplays-qclu_[1, 2]-frateThresh..."
7,kdiba_gor01_one_2006-6-08_14-26-15,laps_time_bin_marginals_df,K:\scratch\collected_outputs\2024-07-09-kdiba_...,None,2024-07-09 00:00:00,"withNewComputedReplays-qclu_[1, 2]-frateThresh..."
8,kdiba_gor01_one_2006-6-08_14-26-15,laps_marginals_df,K:\scratch\collected_outputs\2024-07-09-kdiba_...,None,2024-07-09 00:00:00,"withNewComputedReplays-qclu_[1, 2]-frateThresh..."
9,kdiba_gor01_one_2006-6-08_14-26-15,ripple_time_bin_marginals_df,K:\scratch\collected_outputs\2024-07-09-kdiba_...,None,2024-07-09 00:00:00,


Box(children=(Label(value='across_sessions_parsed_csv_files_path:', layout=Layout(width='auto')), HTML(value="…

Cell executed in: 26.9 ms
Cell executed in: 26.9 ms
Cell executed in: 26.9 ms


In [9]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import export_across_session_CSVs

final_across_session_summary_CSVs_output_path = across_sessions_output_folder.resolve()
display(fullwidth_path_widget(final_across_session_summary_CSVs_output_path, file_name_label='final_across_session_summary_CSVs_output_path:'))
final_csv_export_paths = export_across_session_CSVs(final_output_path=final_across_session_summary_CSVs_output_path, TODAY_DAY_DATE=TODAY_DAY_DATE,
                                                    all_sessions_laps_df=all_sessions_laps_df,  all_sessions_ripple_df=all_sessions_ripple_df,  all_sessions_laps_time_bin_df=all_sessions_laps_time_bin_df,  all_sessions_ripple_time_bin_df=all_sessions_ripple_time_bin_df, 
                                                    all_sessions_simple_pearson_laps_df=all_sessions_simple_pearson_laps_df,  all_sessions_simple_pearson_ripple_df=all_sessions_simple_pearson_ripple_df,
                                                    all_sessions_all_scores_ripple_df=all_sessions_all_scores_ripple_df,  all_sessions_all_scores_laps_df=None,
                                                    )

# final_csv_export_paths: {'AcrossSession_Laps_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_Laps_per-Epoch.csv'),
#  'AcrossSession_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_Ripple_per-Epoch.csv'),
#  'AcrossSession_Laps_per-TimeBin': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_Laps_per-TimeBin.csv'),
#  'AcrossSession_Ripple_per-TimeBin': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_Ripple_per-TimeBin.csv'),
#  'AcrossSession_SimplePearson_Laps_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_SimplePearson_Laps_per-Epoch.csv'),
#  'AcrossSession_SimplePearson_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_SimplePearson_Ripple_per-Epoch.csv'),
#  'AcrossSession_AllScores_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_AllScores_Ripple_per-Epoch.csv')}

if neptuner_run is not None:
    _neptuner_run_parameters = dict(across_sessions_parsed_csv_files_path=across_sessions_parsed_csv_files_path.as_posix(), final_across_session_summary_CSVs_output_path=final_across_session_summary_CSVs_output_path.as_posix(),
                                       )
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'output_files/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing

    for k, v in final_csv_export_paths.items():
        neptuner_run[f"output_files/{k}"].upload(v.resolve().as_posix())
        

final_csv_export_paths

Box(children=(Label(value='final_across_session_summary_CSVs_output_path:', layout=Layout(width='auto')), HTML…

{'AcrossSession_Laps_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-07-09_Apogee_AcrossSession_Laps_per-Epoch.csv'),
 'AcrossSession_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-07-09_Apogee_AcrossSession_Ripple_per-Epoch.csv'),
 'AcrossSession_Laps_per-TimeBin': WindowsPath('K:/scratch/across_sessions/2024-07-09_Apogee_AcrossSession_Laps_per-TimeBin.csv'),
 'AcrossSession_Ripple_per-TimeBin': WindowsPath('K:/scratch/across_sessions/2024-07-09_Apogee_AcrossSession_Ripple_per-TimeBin.csv'),
 'AcrossSession_SimplePearson_Laps_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-07-09_Apogee_AcrossSession_SimplePearson_Laps_per-Epoch.csv'),
 'AcrossSession_SimplePearson_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-07-09_Apogee_AcrossSession_SimplePearson_Ripple_per-Epoch.csv'),
 'AcrossSession_AllScores_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-07-09_Apogee_AcrossSession_AllScores_Ripple_per-Epoch.csv')}

Cell executed in: 384 ms
Cell executed in: 384 ms
Cell executed in: 383 ms


In [ ]:
if (neptuner is not None) and (neptuner_run is not None):
    neptuner.stop()

In [ ]:
parsed_csv_files_df['file_type'].unique()

In [ ]:
parsed_csv_files_df[parsed_csv_files_df['file_type'] == 'ripple_all_scores_merged_df']

### h5_files

In [35]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import load_across_sessions_exported_h5_files

## INPUTS: h5_sessions, session_dict, cuttoff_date, known_bad_session_strs
parsed_h5_files_df, h5_contexts_paths_dict = load_across_sessions_exported_h5_files(collected_outputs_directory=collected_outputs_directory, cuttoff_date=cuttoff_date,
                                                                                    known_bad_session_strs=known_bad_session_strs)
h5_session_contexts = list(h5_contexts_paths_dict.keys())
included_h5_paths = list(h5_contexts_paths_dict.values())

if neptuner_run is not None:
    _neptuner_run_parameters = dict(parsed_h5_files_df=parsed_h5_files_df,
                                       )
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'parsed/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing

parsed_h5_files_df
# h5_contexts_paths_dict


collected_outputs_directory: K:\scratch\collected_outputs


,session,file_type,path,decoding_time_bin_size_str,export_datetime
0,withOldestImportedReplays-qclu,XX,K:\scratch\collected_outputs\2024-07-08_Apogee...,None,2024-07-08
1,"withNormalComputedReplays-qclu_[1,2]-frateThre...",time_bin_size_sweep_results,K:\scratch\collected_outputs\2024-07-08_Apogee...,None,2024-07-08
2,"withNormalComputedReplays-qclu_[1,2]-frateThre...",time_bin_size_sweep_results,K:\scratch\collected_outputs\2024-07-08_Apogee...,None,2024-07-08
3,"withNormalComputedReplays-qclu_[1,2]-frateThre...",time_bin_size_sweep_results,K:\scratch\collected_outputs\2024-07-08_Apogee...,None,2024-07-08
4,"withNewKamranExportedReplays-qclu_[1,2]-frateT...",time_bin_size_sweep_results,K:\scratch\collected_outputs\2024-07-08_Apogee...,None,2024-07-08
5,"withNewKamranExportedReplays-qclu_[1,2]-frateT...",time_bin_size_sweep_results,K:\scratch\collected_outputs\2024-07-08_Apogee...,None,2024-07-08
6,"withNewKamranExportedReplays-qclu_[1,2]-frateT...",time_bin_size_sweep_results,K:\scratch\collected_outputs\2024-07-08_Apogee...,None,2024-07-08
7,"withNewComputedReplays-qclu_[1, 2]-frateThresh...",time_bin_size_sweep_results,K:\scratch\collected_outputs\2024-07-08_Apogee...,None,2024-07-08
8,"withNewComputedReplays-qclu_[1, 2]-frateThresh...",time_bin_size_sweep_results,K:\scratch\collected_outputs\2024-07-08_Apogee...,None,2024-07-08
9,"withNewComputedReplays-qclu_[1, 2]-frateThresh...",time_bin_size_sweep_results,K:\scratch\collected_outputs\2024-07-08_Apogee...,None,2024-07-08


Cell executed in: 4.93 s
Cell executed in: 4.92 s
Cell executed in: 4.91 s


In [11]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsVisualizations

included_session_contexts = deepcopy(h5_session_contexts)
included_h5_paths = deepcopy(included_h5_paths)
num_sessions = len(included_session_contexts)
(neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table), output_path_dicts = AcrossSessionTables.build_and_save_all_combined_tables(included_session_contexts, included_h5_paths,
                                                                                                                                                    override_output_parent_path=across_sessions_output_folder, output_path_suffix=f'{TODAY_DAY_DATE}',
                                                                                                                                                    should_restore_native_column_types=True, include_csv=True, include_pkl=True)


if neptuner_run is not None:
    _neptuner_run_parameters = dict(neuron_identities_table=neuron_identities_table, long_short_fr_indicies_analysis_table=long_short_fr_indicies_analysis_table, neuron_replay_stats_table=neuron_replay_stats_table,
                                       num_sessions=num_sessions)
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'parsed/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing

    for output_name, a_paths_dict in output_path_dicts.items():
        for format_extension, an_output_path in a_paths_dict.items():
            neptuner_run[f"output_files/{format_extension}/{output_name}"].upload(an_output_path.resolve().as_posix())


# {'neuron_identities_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/neuron_identities_table.csv'),
#   '.pkl': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/neuron_identities_table.pkl')},
#  'long_short_fr_indicies_analysis_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/long_short_fr_indicies_analysis_table.csv'),
#   '.pkl': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/long_short_fr_indicies_analysis_table.pkl')},
#  'neuron_replay_stats_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/neuron_replay_stats_table.csv'),
#   '.pkl': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/neuron_replay_stats_table.pkl')}}

output_path_dicts

failed for file path: K:\scratch\collected_outputs\2024-06-12_Apogee_2006-6-12_16-53-46_pipeline_results.h5, table_key: /kdiba/gor01/two/2006-6-12_16-53-46/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/two/2006-6-12_16-53-46/neuron_identities/table``. Skipping.
failed for file path: K:\scratch\collected_outputs\2024-06-12_Apogee_11-02_17-46-44_pipeline_results.h5, table_key: /kdiba/pin01/one/11-02_17-46-44/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/pin01/one/11-02_17-46-44/neuron_identities/table``. Skipping.
concatenating dataframes from 13 of 15 files
failed for file path: K:\scratch\collected_outputs\2024-06-12_Apogee_2006-6-12_16-53-46_pipeline_results.h5, table_key: /kdiba/gor01/two/2006-6-12_16-53-46/global_computations/long_short_fr_indicies_analysis/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/two/2006-6-12_16-53-46/global_computations/long_short_fr_indici

{'neuron_identities_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-07-09_Apogee/neuron_identities_table.csv'),
  '.pkl': WindowsPath('K:/scratch/across_sessions/2024-07-09_Apogee/neuron_identities_table.pkl')},
 'long_short_fr_indicies_analysis_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-07-09_Apogee/long_short_fr_indicies_analysis_table.csv'),
  '.pkl': WindowsPath('K:/scratch/across_sessions/2024-07-09_Apogee/long_short_fr_indicies_analysis_table.pkl')},
 'neuron_replay_stats_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-07-09_Apogee/neuron_replay_stats_table.csv'),
  '.pkl': WindowsPath('K:/scratch/across_sessions/2024-07-09_Apogee/neuron_replay_stats_table.pkl')}}

Cell executed in: 1.33 s
Cell executed in: 1.33 s
Cell executed in: 1.33 s


In [12]:
neuron_identities_table

,neuron_uid,session_uid,session_datetime,format_name,animal,exper_name,session_name,aclu,neuron_type,cluster_index,qclu,shank_index
0,kdiba|gor01|one|2006-6-08_14-26-15|2,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,kdiba,gor01,one,2006-6-08_14-26-15,2,pyr,6,2,1
1,kdiba|gor01|one|2006-6-08_14-26-15|3,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,kdiba,gor01,one,2006-6-08_14-26-15,3,pyr,9,4,1
2,kdiba|gor01|one|2006-6-08_14-26-15|4,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,kdiba,gor01,one,2006-6-08_14-26-15,4,pyr,10,4,1
3,kdiba|gor01|one|2006-6-08_14-26-15|5,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,kdiba,gor01,one,2006-6-08_14-26-15,5,pyr,11,2,1
4,kdiba|gor01|one|2006-6-08_14-26-15|6,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,kdiba,gor01,one,2006-6-08_14-26-15,6,intr,12,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...
781,kdiba|pin01|one|fet11-01_12-58-54|28,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,kdiba,pin01,one,fet11-01_12-58-54,28,pyr,10,2,3
782,kdiba|pin01|one|fet11-01_12-58-54|29,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,kdiba,pin01,one,fet11-01_12-58-54,29,pyr,11,2,3
783,kdiba|pin01|one|fet11-01_12-58-54|30,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,kdiba,pin01,one,fet11-01_12-58-54,30,pyr,14,2,3
784,kdiba|pin01|one|fet11-01_12-58-54|31,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,kdiba,pin01,one,fet11-01_12-58-54,31,pyr,3,4,4


Cell executed in: 13.4 ms
Cell executed in: 12 ms
Cell executed in: 12 ms


In [28]:
neuron_replay_stats_table

,neuron_uid,format_name,animal,exper_name,session_name,neuron_type,aclu,session_uid,session_datetime,long_pf_peak_x,short_pf_peak_x,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,short_replay_mean,replay_diff,long_mean,short_mean,mean_diff,num_replays,long_num_replays,short_num_replays,custom_frs_index,is_rate_extrema,is_refined_exclusive,is_refined_LxC,is_refined_SxC,is_long_peak_left_cap,is_long_peak_right_cap,is_long_peak_either_cap,LS_pf_peak_x_diff,long_LR_pf2D_peak_x,long_LR_pf2D_peak_y,long_RL_pf2D_peak_x,long_RL_pf2D_peak_y,short_LR_pf2D_peak_x,short_LR_pf2D_peak_y,short_RL_pf2D_peak_x,short_RL_pf2D_peak_y,long_LR_pf1D_peak,long_RL_pf1D_peak,short_LR_pf1D_peak,short_RL_pf1D_peak,peak_diff_LR_pf1D_peak,peak_diff_RL_pf1D_peak
0,kdiba|gor01|one|2006-6-08_14-26-15|2,kdiba,gor01,one,2006-6-08_14-26-15,pyr,2,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,NaN,NaN,SHARED,0.41242440259338126,0.11553338629183138,-0.2968910163015499,7.912739600903586,nan,nan,4.162582001748484,nan,nan,45,45,0,0.2334295442830472,False,False,False,False,False,False,False,nan,212.155527,138.787910,82.294630,145.066311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,kdiba|gor01|one|2006-6-08_14-26-15|3,kdiba,gor01,one,2006-6-08_14-26-15,pyr,3,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,141.385524,198.280871,SHARED,0.2740687245121229,1.3945946124809006,1.1205258879687776,9.241991017956833,8.5362581427967,-0.7057328751601322,4.758029871234478,4.965426377638801,0.20739650640432306,91,23,68,-0.6929576266509063,True,False,False,False,False,False,False,-56.8953462153255,140.520949,138.781664,NaN,NaN,121.601684,138.785344,195.336656,142.575786,143.282036,NaN,128.109944,NaN,-15.172092,NaN
2,kdiba|gor01|one|2006-6-08_14-26-15|4,kdiba,gor01,one,2006-6-08_14-26-15,pyr,4,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,NaN,179.315755,RIGHT_ONLY,0.11938205363580597,0.9248959624728122,0.8055139088370062,nan,12.484800687765516,nan,nan,6.704848325119164,nan,87,0,87,-0.9825415552245808,True,True,False,True,False,False,False,nan,NaN,NaN,NaN,NaN,NaN,NaN,177.874096,144.799775,NaN,NaN,NaN,NaN,NaN,NaN
3,kdiba|gor01|one|2006-6-08_14-26-15|5,kdiba,gor01,one,2006-6-08_14-26-15,pyr,5,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,99.662271,202.073894,SHARED,2.3260457715375873,2.0126115307154993,-0.31343424082208804,11.994903850060455,11.315249040961577,-0.6796548090988779,7.1604748107990215,6.663930285838538,-0.4965445249604832,229,126,103,-0.035628934583543,False,False,False,False,False,False,False,-102.41162318758592,234.462951,138.805607,97.174785,140.858022,199.777578,144.348824,111.319822,140.813779,234.314590,93.972736,200.177382,109.144828,-34.137208,15.172092
4,kdiba|gor01|one|2006-6-08_14-26-15|6,kdiba,gor01,one,2006-6-08_14-26-15,intr,6,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,NaN,NaN,SHARED,25.389639299524944,12.75463574941881,-12.635003550106134,105.81199584799322,41.36496231876168,-64.44703352923153,65.60081757375909,27.059799034090243,-38.54101853966884,602,345,257,0.09923680577542646,False,False,False,False,False,False,False,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,kdiba|pin01|one|fet11-01_12-58-54|28,kdiba,pin01,one,fet11-01_12-58-54,pyr,28,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,180.571251,176.844138,SHARED,1.2275172634201952,1.3169564137781686,0.08943915035797345,8.804706579928968,9.155474207048053,0.35076762711908493,5.0161119216745815,5.236215310413111,0.2201033887385293,181,100,81,-0.08756873199979255,False,False,False,False,False,False,False,3.7271132894561845,175.339442,141.003729,186.412849,140.908855,176.615749,140.846341,142.356312,146.959402,55.712956,59.440069,130.255222,100.438316,74.542266,40.998246
782,kdiba|pin01|one|fet11-01_12-58-54|

Cell executed in: 27.9 ms
Cell executed in: 26.9 ms
Cell executed in: 33.9 ms


In [29]:
long_short_fr_indicies_analysis_table

,format_name,animal,exper_name,session_name,index,neuron_uid,session_uid,aclu,x_frs_index,y_frs_index,session_datetime
0,kdiba,gor01,one,2006-6-08_14-26-15,0,kdiba|gor01|one|2006-6-08_14-26-15|2,kdiba|gor01|one|2006-6-08_14-26-15,2,0.950171,1.000000,2006-06-08 14:26:15
1,kdiba,gor01,one,2006-6-08_14-26-15,1,kdiba|gor01|one|2006-6-08_14-26-15|3,kdiba|gor01|one|2006-6-08_14-26-15,3,-0.852388,-0.593507,2006-06-08 14:26:15
2,kdiba,gor01,one,2006-6-08_14-26-15,2,kdiba|gor01|one|2006-6-08_14-26-15|4,kdiba|gor01|one|2006-6-08_14-26-15,4,-0.971940,-1.000000,2006-06-08 14:26:15
3,kdiba,gor01,one,2006-6-08_14-26-15,3,kdiba|gor01|one|2006-6-08_14-26-15|5,kdiba|gor01|one|2006-6-08_14-26-15,5,0.147308,-0.043639,2006-06-08 14:26:15
4,kdiba,gor01,one,2006-6-08_14-26-15,4,kdiba|gor01|one|2006-6-08_14-26-15|6,kdiba|gor01|one|2006-6-08_14-26-15,6,0.443550,0.445497,2006-06-08 14:26:15
...,...,...,...,...,...,...,...,...,...,...,...
823,kdiba,pin01,one,fet11-01_12-58-54,26,kdiba|pin01|one|fet11-01_12-58-54|28,kdiba|pin01|one|fet11-01_12-58-54,28,-0.557444,0.026565,2009-11-01 12:58:54
824,kdiba,pin01,one,fet11-01_12-58-54,27,kdiba|pin01|one|fet11-01_12-58-54|29,kdiba|pin01|one|fet11-01_12-58-54,29,-0.294036,0.247180,2009-11-01 12:58:54
825,kdiba,pin01,one,fet11-01_12-58-54,28,kdiba|pin01|one|fet11-01_12-58-54|30,kdiba|pin01|one|fet11-01_12-58-54,30,-0.249504,0.131576,2009-11-01 12:58:54
826,kdiba,pin01,one,fet11-01_12-58-54,29,kdiba|pin01|one|fet11-01_12-58-54|31,kdiba|pin01|one|fet11-01_12-58-54,31,0.256910,0.257815,2009-11-01 12:58:54


Cell executed in: 19.9 ms
Cell executed in: 16.9 ms
Cell executed in: 17.9 ms


In [30]:
print(list(neuron_replay_stats_table.columns))


['neuron_uid', 'format_name', 'animal', 'exper_name', 'session_name', 'neuron_type', 'aclu', 'session_uid', 'session_datetime', 'long_pf_peak_x', 'short_pf_peak_x', 'track_membership', 'long_non_replay_mean', 'short_non_replay_mean', 'non_replay_diff', 'long_replay_mean', 'short_replay_mean', 'replay_diff', 'long_mean', 'short_mean', 'mean_diff', 'num_replays', 'long_num_replays', 'short_num_replays', 'custom_frs_index', 'is_rate_extrema', 'is_refined_exclusive', 'is_refined_LxC', 'is_refined_SxC', 'is_long_peak_left_cap', 'is_long_peak_right_cap', 'is_long_peak_either_cap', 'LS_pf_peak_x_diff', 'long_LR_pf2D_peak_x', 'long_LR_pf2D_peak_y', 'long_RL_pf2D_peak_x', 'long_RL_pf2D_peak_y', 'short_LR_pf2D_peak_x', 'short_LR_pf2D_peak_y', 'short_RL_pf2D_peak_x', 'short_RL_pf2D_peak_y', 'long_LR_pf1D_peak', 'long_RL_pf1D_peak', 'short_LR_pf1D_peak', 'short_RL_pf1D_peak', 'peak_diff_LR_pf1D_peak', 'peak_diff_RL_pf1D_peak']
Cell executed in: 14 ms
Cell executed in: 12 ms
Cell executed in: 9.97 

In [31]:
from pyphocorehelpers.indexing_helpers import reorder_columns, reorder_columns_relative

## Move the "height" columns to the end
# _neuron_columns_order_dict = dict(zip(['neuron_uid', 'format_name', 'animal', 'exper_name', 'session_name', 'neuron_type', 'aclu', 'session_uid', 'session_datetime'], np.arange(4)+4))

# neuron_replay_stats_table = reorder_columns_relative(neuron_replay_stats_table, column_names=['neuron_uid', 'format_name', 'animal', 'exper_name', 'session_name', 'neuron_type', 'aclu', 'session_uid', 'session_datetime'],
#                                                     relative_mode='end')
neuron_replay_stats_table = reorder_columns_relative(neuron_replay_stats_table, column_names=['neuron_uid', 'format_name', 'animal', 'exper_name', 'session_name', 'neuron_type', 'aclu', 'session_uid', 'session_datetime'],
                                                    relative_mode='start')

neuron_replay_stats_table


,neuron_uid,format_name,animal,exper_name,session_name,neuron_type,aclu,session_uid,session_datetime,long_pf_peak_x,short_pf_peak_x,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,short_replay_mean,replay_diff,long_mean,short_mean,mean_diff,num_replays,long_num_replays,short_num_replays,custom_frs_index,is_rate_extrema,is_refined_exclusive,is_refined_LxC,is_refined_SxC,is_long_peak_left_cap,is_long_peak_right_cap,is_long_peak_either_cap,LS_pf_peak_x_diff,long_LR_pf2D_peak_x,long_LR_pf2D_peak_y,long_RL_pf2D_peak_x,long_RL_pf2D_peak_y,short_LR_pf2D_peak_x,short_LR_pf2D_peak_y,short_RL_pf2D_peak_x,short_RL_pf2D_peak_y,long_LR_pf1D_peak,long_RL_pf1D_peak,short_LR_pf1D_peak,short_RL_pf1D_peak,peak_diff_LR_pf1D_peak,peak_diff_RL_pf1D_peak
0,kdiba|gor01|one|2006-6-08_14-26-15|2,kdiba,gor01,one,2006-6-08_14-26-15,pyr,2,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,NaN,NaN,SHARED,0.41242440259338126,0.11553338629183138,-0.2968910163015499,7.912739600903586,nan,nan,4.162582001748484,nan,nan,45,45,0,0.2334295442830472,False,False,False,False,False,False,False,nan,212.155527,138.787910,82.294630,145.066311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,kdiba|gor01|one|2006-6-08_14-26-15|3,kdiba,gor01,one,2006-6-08_14-26-15,pyr,3,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,141.385524,198.280871,SHARED,0.2740687245121229,1.3945946124809006,1.1205258879687776,9.241991017956833,8.5362581427967,-0.7057328751601322,4.758029871234478,4.965426377638801,0.20739650640432306,91,23,68,-0.6929576266509063,True,False,False,False,False,False,False,-56.8953462153255,140.520949,138.781664,NaN,NaN,121.601684,138.785344,195.336656,142.575786,143.282036,NaN,128.109944,NaN,-15.172092,NaN
2,kdiba|gor01|one|2006-6-08_14-26-15|4,kdiba,gor01,one,2006-6-08_14-26-15,pyr,4,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,NaN,179.315755,RIGHT_ONLY,0.11938205363580597,0.9248959624728122,0.8055139088370062,nan,12.484800687765516,nan,nan,6.704848325119164,nan,87,0,87,-0.9825415552245808,True,True,False,True,False,False,False,nan,NaN,NaN,NaN,NaN,NaN,NaN,177.874096,144.799775,NaN,NaN,NaN,NaN,NaN,NaN
3,kdiba|gor01|one|2006-6-08_14-26-15|5,kdiba,gor01,one,2006-6-08_14-26-15,pyr,5,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,99.662271,202.073894,SHARED,2.3260457715375873,2.0126115307154993,-0.31343424082208804,11.994903850060455,11.315249040961577,-0.6796548090988779,7.1604748107990215,6.663930285838538,-0.4965445249604832,229,126,103,-0.035628934583543,False,False,False,False,False,False,False,-102.41162318758592,234.462951,138.805607,97.174785,140.858022,199.777578,144.348824,111.319822,140.813779,234.314590,93.972736,200.177382,109.144828,-34.137208,15.172092
4,kdiba|gor01|one|2006-6-08_14-26-15|6,kdiba,gor01,one,2006-6-08_14-26-15,intr,6,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,NaN,NaN,SHARED,25.389639299524944,12.75463574941881,-12.635003550106134,105.81199584799322,41.36496231876168,-64.44703352923153,65.60081757375909,27.059799034090243,-38.54101853966884,602,345,257,0.09923680577542646,False,False,False,False,False,False,False,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,kdiba|pin01|one|fet11-01_12-58-54|28,kdiba,pin01,one,fet11-01_12-58-54,pyr,28,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,180.571251,176.844138,SHARED,1.2275172634201952,1.3169564137781686,0.08943915035797345,8.804706579928968,9.155474207048053,0.35076762711908493,5.0161119216745815,5.236215310413111,0.2201033887385293,181,100,81,-0.08756873199979255,False,False,False,False,False,False,False,3.7271132894561845,175.339442,141.003729,186.412849,140.908855,176.615749,140.846341,142.356312,146.959402,55.712956,59.440069,130.255222,100.438316,74.542266,40.998246
782,kdiba|pin01|one|fet11-01_12-58-54|

Cell executed in: 29.9 ms
Cell executed in: 26.9 ms
Cell executed in: 26.9 ms


In [32]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults

# matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

AcrossSessionsResults.post_compute_all_sessions_processing(global_data_root_parent_path=collected_outputs_directory, BATCH_DATE_TO_USE=TODAY_DAY_DATE, plotting_enabled=True)

global_batch_result_inst_fr_file_path: K:\scratch\collected_outputs\across_session_result_long_short_recomputed_inst_firing_rate_2024-07-09_Apogee.pkl


AssertionError: 

Cell executed in: 49.8 ms
Cell executed in: 45.8 ms
Cell executed in: 43.9 ms


In [18]:
global_multi_session_context, _out_aggregate_fig_2 = AcrossSessionsVisualizations.across_sessions_bar_graphs(across_session_inst_fr_computation, num_sessions, enable_tiny_point_labels=False, enable_hover_labels=False)

NameError: name 'across_session_inst_fr_computation' is not defined

Cell executed in: 20.9 ms
Cell executed in: 21.2 ms
Cell executed in: 18.6 ms


In [33]:
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=num_sessions) ## WORKS


register_output_file(output_path: C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\SCRATCH\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2024-07-09\kdiba\{}_15_across_sessions_firing_rate_index.png, ...)
	 saved "file:///C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/SCRATCH/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/2024-07-09/kdiba/%7B%7D_15_across_sessions_firing_rate_index.png"
Cell executed in: 75.8 ms
Cell executed in: 71.8 ms
Cell executed in: 69.8 ms


In [20]:
print(list(all_sessions_all_scores_ripple_df.columns)) # ['start', 'stop', 'label', 'duration', 'is_user_annotated_epoch', 'is_valid_epoch', 'P_LR', 'P_RL', 'P_Long', 'P_Short', 'P_Long_LR', 'score_long_LR', 'velocity_long_LR', 'intercept_long_LR', 'speed_long_LR', 'wcorr_long_LR', 'pearsonr_long_LR', 'travel_long_LR', 'coverage_long_LR', 'jump_long_LR', 'longest_sequence_length_ratio_long_LR', 'direction_change_bin_ratio_long_LR', 'congruent_dir_bins_ratio_long_LR', 'total_congruent_direction_change_long_LR', 'P_Long_RL', 'score_long_RL', 'velocity_long_RL', 'intercept_long_RL', 'speed_long_RL', 'wcorr_long_RL', 'pearsonr_long_RL', 'travel_long_RL', 'coverage_long_RL', 'jump_long_RL', 'longest_sequence_length_ratio_long_RL', 'direction_change_bin_ratio_long_RL', 'congruent_dir_bins_ratio_long_RL', 'total_congruent_direction_change_long_RL', 'P_Short_LR', 'score_short_LR', 'velocity_short_LR', 'intercept_short_LR', 'speed_short_LR', 'wcorr_short_LR', 'pearsonr_short_LR', 'travel_short_LR', 'coverage_short_LR', 'jump_short_LR', 'longest_sequence_length_ratio_short_LR', 'direction_change_bin_ratio_short_LR', 'congruent_dir_bins_ratio_short_LR', 'total_congruent_direction_change_short_LR', 'P_Short_RL', 'score_short_RL', 'velocity_short_RL', 'intercept_short_RL', 'speed_short_RL', 'wcorr_short_RL', 'pearsonr_short_RL', 'travel_short_RL', 'coverage_short_RL', 'jump_short_RL', 'longest_sequence_length_ratio_short_RL', 'direction_change_bin_ratio_short_RL', 'congruent_dir_bins_ratio_short_RL', 'total_congruent_direction_change_short_RL', 'ripple_start_t', 'long_best_travel', 'short_best_travel', 'travel_diff', 'long_best_coverage', 'short_best_coverage', 'coverage_diff', 'long_best_jump', 'short_best_jump', 'jump_diff', 'long_best_longest_sequence_length_ratio', 'short_best_longest_sequence_length_ratio', 'longest_sequence_length_ratio_diff', 'long_best_direction_change_bin_ratio', 'short_best_direction_change_bin_ratio', 'direction_change_bin_ratio_diff', 'long_best_congruent_dir_bins_ratio', 'short_best_congruent_dir_bins_ratio', 'congruent_dir_bins_ratio_diff', 'long_best_total_congruent_direction_change', 'short_best_total_congruent_direction_change', 'total_congruent_direction_change_diff', 'session_name', 'time_bin_size', 'delta_aligned_start_t']
# ['long_best_total_congruent_direction_change', 'short_best_total_congruent_direction_change']
['longest_sequence_length_ratio_diff', 'direction_change_bin_ratio_diff', 'congruent_dir_bins_ratio_diff', 'total_congruent_direction_change_diff']

['start', 'stop', 'label', 'duration', 'is_user_annotated_epoch', 'is_valid_epoch', 'P_LR', 'P_RL', 'P_Long', 'P_Short', 'P_Long_LR', 'congruent_dir_bins_ratio_long_LR', 'coverage_long_LR', 'direction_change_bin_ratio_long_LR', 'integral_second_derivative_long_LR', 'intercept_long_LR', 'jump_long_LR', 'longest_sequence_length_ratio_long_LR', 'pearsonr_long_LR', 'score_long_LR', 'speed_long_LR', 'stddev_of_diff_long_LR', 'total_congruent_direction_change_long_LR', 'total_variation_long_LR', 'travel_long_LR', 'velocity_long_LR', 'wcorr_long_LR', 'P_Long_RL', 'congruent_dir_bins_ratio_long_RL', 'coverage_long_RL', 'direction_change_bin_ratio_long_RL', 'integral_second_derivative_long_RL', 'intercept_long_RL', 'jump_long_RL', 'longest_sequence_length_ratio_long_RL', 'pearsonr_long_RL', 'score_long_RL', 'speed_long_RL', 'stddev_of_diff_long_RL', 'total_congruent_direction_change_long_RL', 'total_variation_long_RL', 'travel_long_RL', 'velocity_long_RL', 'wcorr_long_RL', 'P_Short_LR', 'congru

['longest_sequence_length_ratio_diff',
 'direction_change_bin_ratio_diff',
 'congruent_dir_bins_ratio_diff',
 'total_congruent_direction_change_diff']

Cell executed in: 13 ms
Cell executed in: 13 ms
Cell executed in: 11 ms


In [34]:
all_sessions_simple_pearson_ripple_df # 3138 rows × 24 columns

,P_LR,P_RL,P_Long,P_Short,ripple_idx,ripple_start_t,P_Long_LR,P_Long_RL,P_Short_LR,P_Short_RL,most_likely_decoder_index,start,stop,label,duration,long_LR_pf_peak_x_pearsonr,long_RL_pf_peak_x_pearsonr,short_LR_pf_peak_x_pearsonr,short_RL_pf_peak_x_pearsonr,best_decoder_index,time_bin_size,session_name,delta_aligned_start_t,epoch_idx,long_best_pf_peak_x_pearsonr,short_best_pf_peak_x_pearsonr,wcorr_long_LR,wcorr_long_RL,wcorr_short_LR,wcorr_short_RL,long_best_wcorr,short_best_wcorr,wcorr_abs_diff,pearsonr_abs_diff
0,0.738335,0.261665,0.720620,0.279380,0,1.115443,0.532059,0.188561,0.206276,0.073104,0,1.115443,1.344952,0,0.229509,0.154251,0.026814,0.160988,0.126913,2,0.025,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,-1210.442637,0,0.026814,0.126913,0.385005,0.232037,0.343859,0.063143,0.232037,0.063143,0.168894,-0.100099
1,0.469063,0.530937,0.689776,0.310224,1,1.785661,0.323548,0.366228,0.145515,0.164710,1,1.785661,2.083737,1,0.298076,-0.339917,-0.014000,-0.113174,-0.031180,0,0.025,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,-1209.772420,1,-0.339917,-0.113174,-0.269385,-0.201156,-0.176336,0.164906,-0.269385,-0.176336,0.093050,0.226743
2,0.404113,0.595887,0.601093,0.398907,2,16.521638,0.242909,0.358184,0.161203,0.237703,1,16.521638,16.750471,2,0.228833,-0.715846,-0.179173,-0.728934,-0.601163,2,0.025,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,-1195.036442,2,-0.715846,-0.728934,0.195382,0.093375,0.152241,0.051581,0.195382,0.152241,0.043141,-0.013088
3,0.282790,0.717210,0.630853,0.369147,3,32.411058,0.178399,0.452454,0.104391,0.264756,1,32.411058,32.695894,3,0.284836,0.336561,-0.206557,0.554610,-0.164024,2,0.025,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,-1179.147022,3,0.336561,0.554610,0.121819,-0.054191,0.182492,-0.173343,0.121819,0.182492,-0.060674,-0.218049
4,0.513130,0.486870,0.591657,0.408343,4,40.106783,0.303597,0.288060,0.209533,0.198810,0,40.106783,40.366090,4,0.259307,0.188501,-0.155903,0.000412,-0.061830,0,0.025,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,-1171.451297,4,-0.155903,-0.061830,0.071260,-0.049941,0.049186,-0.078044,-0.049941,-0.078044,-0.028103,0.094073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,0.732105,0.267895,0.223390,0.776610,761,2074.828997,0.163545,0.059845,0.568559,0.208050,2,2074.828997,2075.059674,761,0.230676,0.152644,0.360011,0.161174,0.263367,1,0.025,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,863.270917,761,0.360011,0.263367,0.204793,0.253775,0.221240,-0.015158,0.253775,-0.015158,0.238617,0.096644
762,0.484519,0.515481,0.301443,0.698557,762,2076.104460,0.146055,0.155388,0.338464,0.360093,3,2076.104460,2076.215697,762,0.111237,-0.890654,-0.718953,-0.764031,-0.662235,0,0.025,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,864.546380,762,-0.890654,-0.764031,-0.755752,-0.655410,-0.584171,-0.504885,-0.755752,-0.584171,0.171581,0.126623
763,0.793279,0.206721,0.647421,0.352579,763,2083.585114,0.513586,0.133836,0.279693,0.072886,0,2083.585114,2083.878797,763,0.293683,0.201936,0.175598,0.315191,0.188049,2,0.025,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,872.027034,763,0.175598,0.188049,0.074486,-0.011704,0.076875,-0.030331,-0.011704,-0.030331,-0.018626,-0.012451
764,0.702462,0.297538,0.234744,0.765256,764,2089.449897,0.164899,0.069845,0.537563,0.227693,2,2089.449897,2089.557540,764,0.107643,0.141702,0.484482,0.321135,0.482691,1,0.025,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,877.891817,764,0.484482,0.482691,0.783799,0.729433,0.682206,0.772965,0.729433,0.772965,-0.043532,0.001790


Cell executed in: 26.9 ms
Cell executed in: 27.9 ms
Cell executed in: 25.9 ms


In [22]:
all_sessions_simple_pearson_laps_df # 931 rows × 25 columns

,P_LR,P_RL,P_Long,P_Short,lap_idx,lap_start_t,P_Long_LR,P_Long_RL,P_Short_LR,P_Short_RL,most_likely_decoder_index,start,stop,label,duration,lap_id,lap_dir,maze_id,is_LR_dir,true_decoder_index,is_most_likely_track_identity_Long,is_most_likely_direction_LR,best_decoder_index,long_LR_pf_peak_x_pearsonr,long_RL_pf_peak_x_pearsonr,short_LR_pf_peak_x_pearsonr,short_RL_pf_peak_x_pearsonr,session_name,time_bin_size,delta_aligned_start_t,epoch_idx,long_best_pf_peak_x_pearsonr,short_best_pf_peak_x_pearsonr,wcorr_long_LR,wcorr_long_RL,wcorr_short_LR,wcorr_short_RL,long_best_wcorr,short_best_wcorr,wcorr_abs_diff,pearsonr_abs_diff
0,0.693144,0.306856,0.567954,0.432046,0,5.635867,0.393674,0.174280,0.299470,0.132576,0,5.635867,17.447765,0,11.811898,1,1,0,True,0,True,True,0,-0.720827,-0.430939,-0.654920,-0.458982,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,0.025,-1205.922213,0,-0.430939,-0.458982,-0.415597,-0.194018,-0.302980,-0.192330,-0.194018,-0.192330,0.001688,-0.028042
1,0.296024,0.703976,0.716054,0.283946,1,31.862536,0.211969,0.504085,0.084055,0.199891,1,31.862536,39.770308,1,7.907772,2,0,0,False,1,True,False,1,0.635996,0.708676,0.647859,0.681918,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,0.025,-1179.695544,1,0.635996,0.647859,0.448603,0.603620,0.361398,0.472300,0.448603,0.361398,0.087205,-0.011864
2,0.702915,0.297085,0.605743,0.394257,2,135.801698,0.425786,0.179957,0.277129,0.117128,0,135.801698,144.175555,2,8.373857,3,1,0,True,0,True,True,0,-0.699018,-0.505056,-0.637711,-0.472964,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,0.025,-1075.756382,2,-0.505056,-0.472964,-0.513035,-0.238619,-0.356660,-0.221865,-0.238619,-0.221865,0.016754,0.032093
3,0.258971,0.741029,0.745927,0.254073,3,161.458825,0.193173,0.552753,0.065798,0.188276,1,161.458825,167.331790,3,5.872965,4,0,0,False,1,True,False,1,0.607422,0.772204,0.619712,0.751093,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,0.025,-1050.099255,3,0.607422,0.619712,0.478444,0.673764,0.422847,0.521091,0.478444,0.422847,0.055597,-0.012289
4,0.701744,0.298256,0.546189,0.453811,4,234.465983,0.383285,0.162904,0.318459,0.135352,0,234.465983,239.806649,4,5.340666,5,1,0,True,0,True,True,0,-0.585466,-0.486812,-0.498981,-0.504781,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,0.025,-977.092097,4,-0.486812,-0.504781,-0.306792,-0.127054,-0.166913,-0.111554,-0.127054,-0.111554,0.015500,-0.017968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.296302,0.703698,0.343282,0.656718,75,2002.883510,0.101715,0.241567,0.194587,0.462132,3,2002.883510,2008.287350,75,5.403840,76,0,1,False,3,False,False,2,0.597626,0.600286,0.645345,0.627770,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,0.025,791.325430,75,0.597626,0.645345,0.449619,0.505572,0.390434,0.482695,0.449619,0.390434,0.059184,-0.047719
76,0.714896,0.285104,0.372068,0.627932,76,2019.131459,0.265990,0.106078,0.448906,0.179026,2,2019.131459,2024.905445,76,5.773986,77,1,1,True,2,False,True,2,-0.616940,-0.493212,-0.653806,-0.534066,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,0.025,807.573379,76,-0.493212,-0.534066,-0.454784,-0.160041,-0.308124,-0.198984,-0.160041,-0.198984,-0.038943,-0.040854
77,0.272559,0.727441,0.381900,0.618100,77,2026.006596,0.104091,0.277810,0.168469,0.449631,3,2026.006596,2031.378739,77,5.372143,78,0,1,False,3,False,False,1,0.673396,0.740802,0.670267,0.739416,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,0.025,814.448516,77,0.673396,0.670267,0.425603,0.551272,0.387497,0.502186,0.425603,0.387497,0.038106,0.003129
78,0.740742,0.259258,0.370782,0.629218,78,2037.617503,0.274654,0.096128,0.466089,0.163130,2,2037.617503,2042.288964,78,4.671461,79,1,1,True,2,False,True,2,-0.658931,-0.426664,-0.717689,-0.482826,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,0.025,826.059423,78,-0.426664,-0.482826,-0.454993,-0.152039,-0.365656,-0.215280,-0.152039,-0.215280,-0.063242,-0.056161


Cell executed in: 26.9 ms
Cell executed in: 23.9 ms
Cell executed in: 22.9 ms


In [23]:
all_sessions_ripple_df # 17592 rows × 10 columns

,P_LR,P_RL,P_Long,P_Short,ripple_idx,ripple_start_t,session_name,delta_aligned_start_t,time_bin_size,epoch_idx
0,0.862064,0.137936,0.898964,0.101036,0,10.190894,kdiba_gor01_one_2006-6-08_14-26-15__withNewCom...,-1201.367186,0.02,0
1,0.870587,0.129413,0.474199,0.525801,1,12.543953,kdiba_gor01_one_2006-6-08_14-26-15__withNewCom...,-1199.014127,0.02,1
2,0.720703,0.279297,0.623205,0.376795,2,13.770448,kdiba_gor01_one_2006-6-08_14-26-15__withNewCom...,-1197.787632,0.02,2
3,0.784330,0.215670,0.598649,0.401351,3,14.376062,kdiba_gor01_one_2006-6-08_14-26-15__withNewCom...,-1197.182018,0.02,3
4,0.245822,0.754178,0.655021,0.344979,4,31.241763,kdiba_gor01_one_2006-6-08_14-26-15__withNewCom...,-1180.316317,0.02,4
...,...,...,...,...,...,...,...,...,...,...
4684,0.588176,0.411824,0.515931,0.484069,347,2556.119808,kdiba_gor01_two_2006-6-07_16-40-19,1319.853563,0.02,347
4685,0.078436,0.921564,0.290239,0.709761,348,2568.836047,kdiba_gor01_two_2006-6-07_16-40-19,1332.569802,0.02,348
4686,0.246361,0.753639,0.349811,0.650189,349,2569.505407,kdiba_gor01_two_2006-6-07_16-40-19,1333.239161,0.02,349
4687,0.003637,0.996363,0.477797,0.522203,350,2586.495761,kdiba_gor01_two_2006-6-07_16-40-19,1350.229515,0.02,350


Cell executed in: 18.9 ms
Cell executed in: 18.9 ms
Cell executed in: 17.9 ms


In [24]:
csv_files

[WindowsPath('K:/scratch/collected_outputs/2024-05-07_0625PM-kdiba_vvp01_two_2006-4-10_12-58-3-(laps_marginals_df)_tbin-0.25.csv'),
 WindowsPath('K:/scratch/collected_outputs/2024-05-07_0625PM-kdiba_vvp01_two_2006-4-10_12-58-3-(ripple_time_bin_marginals_df)_tbin-0.025.csv'),
 WindowsPath('K:/scratch/collected_outputs/2024-05-07_0625PM-kdiba_vvp01_two_2006-4-10_12-58-3-(ripple_marginals_df)_tbin-0.025.csv'),
 WindowsPath('K:/scratch/collected_outputs/2024-05-07_0620PM-kdiba_vvp01_two_2006-4-10_12-58-3-(laps_weighted_corr_merged_df)_tbin-0.25.csv'),
 WindowsPath('K:/scratch/collected_outputs/2024-05-07_0620PM-kdiba_vvp01_two_2006-4-10_12-58-3-(ripple_weighted_corr_merged_df)_tbin-0.025.csv'),
 WindowsPath('K:/scratch/collected_outputs/2024-05-07_0620PM-kdiba_vvp01_two_2006-4-10_12-58-3-(laps_simple_pf_pearson_merged_df)_tbin-0.25.csv'),
 WindowsPath('K:/scratch/collected_outputs/2024-05-07_0620PM-kdiba_vvp01_two_2006-4-10_12-58-3-(ripple_simple_pf_pearson_merged_df)_tbin-0.025.csv'),
 Wi

Cell executed in: 22.9 ms
Cell executed in: 23.9 ms
Cell executed in: 21.9 ms


In [25]:
all_sessions_all_scores_ripple_df

,start,stop,label,duration,is_user_annotated_epoch,is_valid_epoch,P_LR,P_RL,P_Long,P_Short,P_Long_LR,congruent_dir_bins_ratio_long_LR,coverage_long_LR,direction_change_bin_ratio_long_LR,integral_second_derivative_long_LR,intercept_long_LR,jump_long_LR,longest_sequence_length_ratio_long_LR,pearsonr_long_LR,score_long_LR,speed_long_LR,stddev_of_diff_long_LR,total_congruent_direction_change_long_LR,total_variation_long_LR,travel_long_LR,velocity_long_LR,wcorr_long_LR,P_Long_RL,congruent_dir_bins_ratio_long_RL,coverage_long_RL,direction_change_bin_ratio_long_RL,integral_second_derivative_long_RL,intercept_long_RL,jump_long_RL,longest_sequence_length_ratio_long_RL,pearsonr_long_RL,score_long_RL,speed_long_RL,stddev_of_diff_long_RL,total_congruent_direction_change_long_RL,total_variation_long_RL,travel_long_RL,velocity_long_RL,wcorr_long_RL,P_Short_LR,congruent_dir_bins_ratio_short_LR,coverage_short_LR,direction_change_bin_ratio_short_LR,integral_second_derivative_short_LR,intercept_short_LR,...,short_best_score,score_diff,long_best_velocity,short_best_velocity,velocity_diff,long_best_intercept,short_best_intercept,intercept_diff,long_best_speed,short_best_speed,speed_diff,long_best_wcorr,short_best_wcorr,wcorr_diff,long_best_pearsonr,short_best_pearsonr,pearsonr_diff,long_best_travel,short_best_travel,travel_diff,long_best_coverage,short_best_coverage,coverage_diff,long_best_jump,short_best_jump,jump_diff,long_best_longest_sequence_length_ratio,short_best_longest_sequence_length_ratio,longest_sequence_length_ratio_diff,long_best_direction_change_bin_ratio,short_best_direction_change_bin_ratio,direction_change_bin_ratio_diff,long_best_congruent_dir_bins_ratio,short_best_congruent_dir_bins_ratio,congruent_dir_bins_ratio_diff,long_best_total_congruent_direction_change,short_best_total_congruent_direction_change,total_congruent_direction_change_diff,long_best_total_variation,short_best_total_variation,total_variation_diff,long_best_integral_second_derivative,short_best_integral_second_derivative,integral_second_derivative_diff,long_best_stddev_of_diff,short_best_stddev_of_diff,stddev_of_diff_diff,session_name,time_bin_size,delta_aligned_start_t
0,1.115443,1.344952,0,0.229509,False,True,0.738335,0.261665,0.720620,0.279380,0.532059,0.500000,0.157895,0.250000,231069.993960,143.0588068181242,0.004339,0.666667,0.154251,0.481135,75.86046162043404,97.912861,227.581385,428.611608,0.250357,-75.86046162043404,0.385005,0.188561,0.625000,0.491228,0.750000,664277.676430,88.89661331640616,0.004339,0.222222,0.026814,0.174318,113.79069243065106,162.923477,720.674385,1259.283663,0.735563,-113.79069243065106,0.232037,0.206276,0.625000,0.175439,0.250000,149322.923063,83.73701387858063,...,0.185801,-0.011483,-113.79069243065106,-132.7558078357595,-18.965115405108435,88.89661331640616,37.16086653890616,51.7357467775,113.79069243065106,132.7558078357595,-18.965115405108435,0.232037,0.063143,0.168894,0.026814,0.126913,-0.100099,0.735563,0.935086,-0.199523,0.491228,0.280702,0.210526,0.004339,0.005952,-0.001613,0.222222,0.222222,0.000000,0.750000,0.750000,0.000,0.625000,0.625000,0.000000,720.674385,553.781370,166.893016,1259.283663,1077.218555,182.065108,664277.676430,548548.454624,115729.221805,162.923477,145.599725,17.323752,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,0.025,-1210.442637
1,1.785661,2.083737,1,0.298076,False,True,0.469063,0.530937,0.689776,0.310224,0.323548,0.500000,0.140351,0.200000,226667.564588,274.8720282545389,0.004423,0.454545,-0.339917,0.553294,15.172092324086748,88.452290,227.581385,443.783700,0.207376,15.172092324086748,-0.269385,0.366228,0.100000,0.280702,0.300000,271483.144638,380.20643522951934,0.003922,0.363636,-0.014000,0.258892,75.86046162043408,111.581946,0.000000,804.120893,0.375757,75.86046162043408,-0.201156,0.145515,0.600000,0.157895,0.400000,159235.582663,147.5101787402772,...,0.520414,0.032880,15.172092324086748,-30.34418464817361,-15.172092324086861,274.8720282545389,147.5101787402772,127.3618495142617,15.172092324

Cell executed in: 42.9 ms
Cell executed in: 42.9 ms
Cell executed in: 40.9 ms


In [26]:

all_sessions_simple_pearson_ripple_df

,P_LR,P_RL,P_Long,P_Short,ripple_idx,ripple_start_t,P_Long_LR,P_Long_RL,P_Short_LR,P_Short_RL,most_likely_decoder_index,start,stop,label,duration,long_LR_pf_peak_x_pearsonr,long_RL_pf_peak_x_pearsonr,short_LR_pf_peak_x_pearsonr,short_RL_pf_peak_x_pearsonr,best_decoder_index,time_bin_size,session_name,delta_aligned_start_t,epoch_idx,long_best_pf_peak_x_pearsonr,short_best_pf_peak_x_pearsonr,wcorr_long_LR,wcorr_long_RL,wcorr_short_LR,wcorr_short_RL,long_best_wcorr,short_best_wcorr,wcorr_abs_diff,pearsonr_abs_diff
0,0.738335,0.261665,0.720620,0.279380,0,1.115443,0.532059,0.188561,0.206276,0.073104,0,1.115443,1.344952,0,0.229509,0.154251,0.026814,0.160988,0.126913,2,0.025,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,-1210.442637,0,0.026814,0.126913,0.385005,0.232037,0.343859,0.063143,0.232037,0.063143,0.168894,-0.100099
1,0.469063,0.530937,0.689776,0.310224,1,1.785661,0.323548,0.366228,0.145515,0.164710,1,1.785661,2.083737,1,0.298076,-0.339917,-0.014000,-0.113174,-0.031180,0,0.025,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,-1209.772420,1,-0.339917,-0.113174,-0.269385,-0.201156,-0.176336,0.164906,-0.269385,-0.176336,0.093050,0.226743
2,0.404113,0.595887,0.601093,0.398907,2,16.521638,0.242909,0.358184,0.161203,0.237703,1,16.521638,16.750471,2,0.228833,-0.715846,-0.179173,-0.728934,-0.601163,2,0.025,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,-1195.036442,2,-0.715846,-0.728934,0.195382,0.093375,0.152241,0.051581,0.195382,0.152241,0.043141,-0.013088
3,0.282790,0.717210,0.630853,0.369147,3,32.411058,0.178399,0.452454,0.104391,0.264756,1,32.411058,32.695894,3,0.284836,0.336561,-0.206557,0.554610,-0.164024,2,0.025,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,-1179.147022,3,0.336561,0.554610,0.121819,-0.054191,0.182492,-0.173343,0.121819,0.182492,-0.060674,-0.218049
4,0.513130,0.486870,0.591657,0.408343,4,40.106783,0.303597,0.288060,0.209533,0.198810,0,40.106783,40.366090,4,0.259307,0.188501,-0.155903,0.000412,-0.061830,0,0.025,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,-1171.451297,4,-0.155903,-0.061830,0.071260,-0.049941,0.049186,-0.078044,-0.049941,-0.078044,-0.028103,0.094073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,0.732105,0.267895,0.223390,0.776610,761,2074.828997,0.163545,0.059845,0.568559,0.208050,2,2074.828997,2075.059674,761,0.230676,0.152644,0.360011,0.161174,0.263367,1,0.025,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,863.270917,761,0.360011,0.263367,0.204793,0.253775,0.221240,-0.015158,0.253775,-0.015158,0.238617,0.096644
762,0.484519,0.515481,0.301443,0.698557,762,2076.104460,0.146055,0.155388,0.338464,0.360093,3,2076.104460,2076.215697,762,0.111237,-0.890654,-0.718953,-0.764031,-0.662235,0,0.025,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,864.546380,762,-0.890654,-0.764031,-0.755752,-0.655410,-0.584171,-0.504885,-0.755752,-0.584171,0.171581,0.126623
763,0.793279,0.206721,0.647421,0.352579,763,2083.585114,0.513586,0.133836,0.279693,0.072886,0,2083.585114,2083.878797,763,0.293683,0.201936,0.175598,0.315191,0.188049,2,0.025,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,872.027034,763,0.175598,0.188049,0.074486,-0.011704,0.076875,-0.030331,-0.011704,-0.030331,-0.018626,-0.012451
764,0.702462,0.297538,0.234744,0.765256,764,2089.449897,0.164899,0.069845,0.537563,0.227693,2,2089.449897,2089.557540,764,0.107643,0.141702,0.484482,0.321135,0.482691,1,0.025,kdiba_gor01_one_2006-6-08_14-26-15__withNewKam...,877.891817,764,0.484482,0.482691,0.783799,0.729433,0.682206,0.772965,0.729433,0.772965,-0.043532,0.001790


Cell executed in: 28.9 ms
Cell executed in: 25.9 ms
Cell executed in: 24.7 ms


In [27]:
## find all columns starting with 'wcorr': wcorr_long_LR
sub_string: str = 'wcorr'
sub_string: str = 'pearsonr'
columns_list = list(all_sessions_simple_pearson_ripple_df.columns)
matching_columns = [s for s in columns_list if sub_string in s]
print(matching_columns) # ['wcorr_long_LR', 'wcorr_long_RL', 'wcorr_short_LR', 'wcorr_short_RL', 'long_best_wcorr', 'short_best_wcorr', 'wcorr_abs_diff']

['long_LR_pf_peak_x_pearsonr', 'long_RL_pf_peak_x_pearsonr', 'short_LR_pf_peak_x_pearsonr', 'short_RL_pf_peak_x_pearsonr', 'long_best_pf_peak_x_pearsonr', 'short_best_pf_peak_x_pearsonr', 'pearsonr_abs_diff']
['wcorr_long_LR', 'wcorr_long_RL', 'wcorr_short_LR', 'wcorr_short_RL', 'long_best_wcorr', 'short_best_wcorr', 'wcorr_abs_diff']

assert np.shape(all_sessions_simple_pearson_ripple_df)[0] == np.shape(all_sessions_all_scores_ripple_df)[0], f"np.shape(all_sessions_all_scores_ripple_df)[0]: {np.shape(all_sessions_all_scores_ripple_df)[0]} != np.shape(all_sessions_simple_pearson_ripple_df)[0]: {np.shape(all_sessions_simple_pearson_ripple_df)[0]}"



['long_LR_pf_peak_x_pearsonr', 'long_RL_pf_peak_x_pearsonr', 'short_LR_pf_peak_x_pearsonr', 'short_RL_pf_peak_x_pearsonr', 'long_best_pf_peak_x_pearsonr', 'short_best_pf_peak_x_pearsonr', 'pearsonr_abs_diff']
Cell executed in: 13 ms
Cell executed in: 9.97 ms
Cell executed in: 8.97 ms


In [ ]:
['long_LR_pf_peak_x_pearsonr', 'long_RL_pf_peak_x_pearsonr', 'short_LR_pf_peak_x_pearsonr', 'short_RL_pf_peak_x_pearsonr', 'long_best_pf_peak_x_pearsonr', 'short_best_pf_peak_x_pearsonr', 'pearsonr_abs_diff']
['wcorr_long_LR', 'wcorr_long_RL', 'wcorr_short_LR', 'wcorr_short_RL', 'long_best_wcorr', 'short_best_wcorr', 'wcorr_abs_diff']

# 2024-03-02 - Get only the user-annotated ripples:

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import _split_user_annotated_ripple_df

## Bump
# input_df = all_sessions_simple_pearson_ripple_df
# input_df = all_sessions_all_scores_ripple_df

all_sessions_all_scores_ripple_df, (valid_ripple_df, invalid_ripple_df), (user_approved_ripple_df, user_rejected_ripple_df) = _split_user_annotated_ripple_df(all_sessions_all_scores_ripple_df)

## 2024-03-14 - 'is_valid_epoch' column
# 'is_valid_epoch'
## OUTPUTS: valid_ripple_df, invalid_ripple_df, user_approved_ripple_df, user_rejected_ripple_df, (user_annotated_epoch_unique_session_names, unannotated_session_names)
user_approved_ripple_df

### 2024-02-29 - 4pm - Filter the events for those meeting wcorr criteria:


In [ ]:
# df: pd.DataFrame = deepcopy(all_sessions_user_annotated_ripple_df)
df: pd.DataFrame = deepcopy(valid_ripple_df) # valid epochs, but not just those that the user approved
# df: pd.DataFrame = deepcopy(user_approved_ripple_df)

## INPUTS: df

min_wcorr_threshold: float = 0.33
min_wcorr_diff_threshold: float = 0.2

# is_included_large_wcorr_diff = np.any((df[['wcorr_abs_diff']].abs() > min_wcorr_diff_threshold), axis=1)
is_included_large_wcorr_diff = np.any((df[['wcorr_diff']].abs() > min_wcorr_diff_threshold), axis=1)
is_included_high_wcorr = np.any((df[['long_best_wcorr', 'short_best_wcorr']].abs() > min_wcorr_threshold), axis=1)

df = df[is_included_high_wcorr]
df

# wcorr_long_LR


In [ ]:
all_sessions_all_scores_ripple_df.time_bin_size.unique()

In [ ]:
all_sessions_ripple_time_bin_df

### Single-time bin size version:

In [ ]:
from neuropy.utils.misc import add_explicit_dataframe_columns_from_lookup_df


# target_time_bin_size: float = 0.005 # 0.025 # 0.08222222

target_time_bin_size: float = 0.025

## INPUTS: all_sessions_all_scores_ripple_df, (all_sessions_ripple_df, all_sessions_ripple_time_bin_df)

def build_single_time_bin_size_dfs(all_sessions_all_scores_epochs_df, all_sessions_epochs_df, all_sessions_epochs_time_bin_df, target_time_bin_size: float, included_columns = ['delta_aligned_start_t', 'is_user_annotated_epoch', 'is_valid_epoch']):
    """ Filters the epochs dataframe down to a single time_bin_size specified by `target_time_bin_size`.  """
    print(f'all_sessions_ripple_df.time_bin_size.unique(): {all_sessions_epochs_df.time_bin_size.unique()}')
    single_time_bin_size_all_sessions_epochs_df = deepcopy(all_sessions_epochs_df[np.isclose(all_sessions_epochs_df['time_bin_size'], target_time_bin_size)])
    print(f'np.shape(single_time_bin_size_all_sessions_ripple_df): {np.shape(single_time_bin_size_all_sessions_epochs_df)}')

    print(f'all_sessions_ripple_time_bin_df.time_bin_size.unique(): {all_sessions_epochs_time_bin_df.time_bin_size.unique()}')
    single_time_bin_size_all_sessions_epochs_time_bin_df = deepcopy(all_sessions_epochs_time_bin_df[np.isclose(all_sessions_epochs_time_bin_df['time_bin_size'], target_time_bin_size)])
    print(f'np.shape(single_time_bin_size_all_sessions_ripple_time_bin_df): {np.shape(single_time_bin_size_all_sessions_epochs_time_bin_df)}')

    # single_time_bin_size_all_sessions_ripple_time_bin_df
    # single_time_bin_size_all_sessions_ripple_df # has ['ripple_start_t']
    # all_sessions_all_scores_ripple_df

    ## recover the important columns (user-annotation, epoch validity) from the newer `all_sessions_all_scores_ripple_df` for use in 'single_time_bin_size_all_sessions_ripple_df'
    all_sessions_all_scores_epochs_df['delta_aligned_start_t'] = all_sessions_all_scores_epochs_df['delta_aligned_start_t'].astype(float)
    single_time_bin_size_all_sessions_epochs_df['delta_aligned_start_t'] = single_time_bin_size_all_sessions_epochs_df['delta_aligned_start_t'].astype(float)

    # Added 'delta_aligned_start_t' for the merge
    single_time_bin_size_all_sessions_epochs_df = add_explicit_dataframe_columns_from_lookup_df(single_time_bin_size_all_sessions_epochs_df, all_sessions_all_scores_epochs_df[included_columns], join_column_name='delta_aligned_start_t')
    single_time_bin_size_all_sessions_epochs_df.sort_values(by=['delta_aligned_start_t'], inplace=True) # Need to re-sort by timestamps once done
    single_time_bin_size_all_sessions_epochs_df

    single_time_bin_size_all_sessions_epochs_time_bin_df = add_explicit_dataframe_columns_from_lookup_df(single_time_bin_size_all_sessions_epochs_time_bin_df, all_sessions_all_scores_epochs_df[included_columns], join_column_name='delta_aligned_start_t')
    single_time_bin_size_all_sessions_epochs_time_bin_df.sort_values(by=['t_bin_center'], inplace=True) # Need to re-sort by timestamps once done
    
    ## Add plotly helper columns:
    for a_df in (all_sessions_all_scores_epochs_df, all_sessions_epochs_df, all_sessions_epochs_time_bin_df, single_time_bin_size_all_sessions_epochs_df, single_time_bin_size_all_sessions_epochs_time_bin_df):
        a_df['pre_post_delta_category'] = 'post-delta'
        a_df['pre_post_delta_category'][a_df['delta_aligned_start_t'] < 0.0] = 'pre-delta'

    ## OUTPUTS: single_time_bin_size_all_sessions_ripple_df, single_time_bin_size_all_sessions_ripple_time_bin_df
    return single_time_bin_size_all_sessions_epochs_df, single_time_bin_size_all_sessions_epochs_time_bin_df


single_time_bin_size_all_sessions_ripple_df, single_time_bin_size_all_sessions_ripple_time_bin_df = build_single_time_bin_size_dfs(all_sessions_all_scores_epochs_df=all_sessions_all_scores_ripple_df,
                                                                                                            all_sessions_epochs_df=all_sessions_ripple_df, all_sessions_epochs_time_bin_df=all_sessions_ripple_time_bin_df, target_time_bin_size=0.005,
                                                                                                            included_columns = ['delta_aligned_start_t', 'is_user_annotated_epoch', 'is_valid_epoch'])

                                                                                                            
single_time_bin_size_all_sessions_laps_df, single_time_bin_size_all_sessions_laps_time_bin_df = build_single_time_bin_size_dfs(all_sessions_all_scores_epochs_df=all_sessions_simple_pearson_laps_df,
                                                                                                            all_sessions_epochs_df=all_sessions_laps_df, all_sessions_epochs_time_bin_df=all_sessions_laps_time_bin_df, target_time_bin_size=0.005,
                                                                                                            included_columns = ['delta_aligned_start_t',])

## OUTPUTS: single_time_bin_size_all_sessions_ripple_df, single_time_bin_size_all_sessions_ripple_time_bin_df
## OUTPUTS: single_time_bin_size_all_sessions_laps_df, single_time_bin_size_all_sessions_laps_time_bin_df 

single_time_bin_size_all_sessions_laps_df


# # Recover the important columns from the newer `all_sessions_all_scores_ripple_df`
# included_columns = ['delta_aligned_start_t', 'is_user_annotated_epoch', 'is_valid_epoch']  # Added 'delta_aligned_start_t' for the merge
# single_time_bin_size_all_sessions_ripple_df = pd.merge(single_time_bin_size_all_sessions_ripple_df, 
#                      all_sessions_all_scores_ripple_df[included_columns], 
#                      on='delta_aligned_start_t', 
#                      how='left')


# single_time_bin_size_all_sessions_ripple_df
## OUTPUTS: single_time_bin_size_all_sessions_ripple_df, single_time_bin_size_all_sessions_ripple_df
# single_time_bin_size_all_sessions_ripple_time_bin_df
all_sessions_simple_pearson_laps_df
single_time_bin_size_all_sessions_laps_time_bin_df

In [ ]:
# INPUTS: all_sessions_all_scores_ripple_df, (all_sessions_ripple_df, all_sessions_ripple_time_bin_df)
all_sessions_ripple_df

In [ ]:
all_sessions_ripple_time_bin_df

In [ ]:
all_sessions_all_scores_ripple_df.delta_aligned_start_t.nunique()
all_sessions_all_scores_ripple_df.start.nunique()
all_sessions_all_scores_ripple_df

In [ ]:
single_time_bin_size_all_sessions_ripple_time_bin_df.delta_aligned_start_t.nunique()

In [ ]:
print(all_sessions_ripple_time_bin_df.delta_aligned_start_t.nunique())
print(all_sessions_ripple_time_bin_df.t_bin_center.nunique())

# Plotting via Plotly
`!pip install kaleido=="v0.1.0.post1" `

In [ ]:
import plotly.io as pio
from pyphoplacecellanalysis.Pho2D.plotly.plotly_templates import PlotlyHelpers
from pyphoplacecellanalysis.Pho2D.statistics_plotting_helpers import plot_histograms_across_sessions, plot_stacked_histograms
from pyphoplacecellanalysis.Pho2D.plotly.Extensions.plotly_helpers import plotly_helper_save_figures, _helper_build_figure, plotly_pre_post_delta_scatter, plot_across_sessions_scatter_results

# fig_size_kwargs = {'width': 1650, 'height': 480}
resolution_multiplier = 2
fig_size_kwargs = {'width': resolution_multiplier*1650, 'height': resolution_multiplier*480}
is_dark_mode, template = PlotlyHelpers.get_plotly_template(is_dark_mode=False)
pio.templates.default = template

# figure_export_path = Path(r'E:\Dropbox (Personal)\Active\Kamran Diba Lab\Presentations\2024-05-30 - Pho iNAV Poster\Figures').resolve()
# figure_export_path = Path('/Users/pho/Dropbox (Personal)/Active/Kamran Diba Lab/Presentations/2024-05-30 - Pho iNAV Poster/Figures').resolve()

# assert figure_export_path.exists()

from PIL import Image

from pyphocorehelpers.programming_helpers import copy_image_to_clipboard

def save_plotly(a_fig, a_fig_context):
    """ 
    captures: TODAY_DAY_DATE, figures_folder, neptuner_run
    """
    fig_save_path: Path = figures_folder.joinpath('_'.join([TODAY_DAY_DATE, sanitize_filename_for_Windows(a_fig_context.get_description())])).resolve()
    figure_out_paths = {'.html': fig_save_path.with_suffix('.html'), '.png': fig_save_path.with_suffix('.png')}
    a_fig.write_html(figure_out_paths['.html'])
    display(fullwidth_path_widget(figure_out_paths['.html'], file_name_label='.html'))
    # print(file_uri_from_path(figure_out_paths['.html']))
    a_fig.write_image(figure_out_paths['.png'])
    # print(file_uri_from_path(figure_out_paths['.png']))
    display(fullwidth_path_widget(figure_out_paths['.png'], file_name_label='.png'))

    if neptuner_run is not None:
        a_full_figure_path_key: str = a_fig_context.get_description(separator='/', include_property_names=True, key_value_separator=':') # .replace(' ', '_')
        # a_full_figure_path_key: str = a_fig_context.get_description(separator=':', include_property_names=True, key_value_separator='|')
        print(f'a_full_figure_path_key: "{a_full_figure_path_key}"')
        # neptuner_run['outputs']['figures'][f"{a_full_figure_path_key}"].upload(a_fig)
        neptuner_run['outputs']['figures'][f"{a_full_figure_path_key}"].upload(figure_out_paths['.html'].as_posix())
        # neptuner.figures[f"{a_full_figure_path_key}"].upload(a_fig)
        
    return figure_out_paths


In [ ]:
## INPUTS: all_sessions_all_scores_ripple_df
# concatenated_ripple_df = deepcopy(all_sessions_all_scores_ripple_df).infer_objects()
# concatenated_ripple_df = deepcopy(single_time_bin_size_all_sessions_ripple_df).infer_objects() ## single time bin size
parsed_csv_files_df[parsed_csv_files_df['replay_epoch_names'] == 'withNewKamranExportedReplays']


concatenated_ripple_df = deepcopy(single_time_bin_size_all_sessions_ripple_df).infer_objects()


# single_time_bin_size_all_sessions_laps_df, single_time_bin_size_all_sessions_laps_time_bin_df

concatenated_ripple_df

In [ ]:


ripple_title_prefix = "User Annotated Sessions Ripples"

# concatenated_ripple_df = deepcopy(user_approved_ripple_df)
# ripple_title_prefix = "User Selected Ripples Only"

# concatenated_ripple_df = deepcopy(user_rejected_ripple_df)
# ripple_title_prefix = "User Rejected Ripples Only"

# concatenated_ripple_df = deepcopy(df)


# Create a bubble chart for ripples
enabled_time_bin_sizes = None
ripple_num_unique_sessions: int = concatenated_ripple_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
ripple_num_unique_time_bins: int = concatenated_ripple_df.time_bin_size.nunique(dropna=True)
ripple_title_string_suffix: str = f'{ripple_num_unique_sessions} Sessions'
ripple_title: str = f"{ripple_title_prefix} - {ripple_title_string_suffix}"
fig_ripples, figure_ripples_context = _helper_build_figure(data_results_df=concatenated_ripple_df, histogram_bins=25, earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end, enabled_time_bin_sizes=enabled_time_bin_sizes, main_plot_mode='default', title=ripple_title, is_dark_mode=is_dark_mode)
figure_ripples_context = figure_ripples_context.adding_context_if_missing(num_sessions=num_sessions, plot_type='hist', comparison='pre-post-delta')
fig_ripples = fig_ripples.update_layout(fig_size_kwargs)
fig_ripples.show()
# fig_to_clipboard(fig_ripples)
figure_out_paths = save_plotly(a_fig=fig_ripples, a_fig_context=figure_ripples_context)

In [ ]:
histogram_bins = 25
# concatenated_ripple_df = deepcopy(df)

# ripple_title_prefix = 'test'
# # Create a bubble chart for ripples
# ripple_num_unique_sessions: int = concatenated_ripple_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
# # ripple_num_unique_time_bins: int = concatenated_ripple_df.time_bin_size.nunique(dropna=True)
# ripple_title_string_suffix: str = f'{ripple_num_unique_sessions} Sessions'
# ripple_title: str = f"{ripple_title_prefix} - {ripple_title_string_suffix}"
# fig_ripples = _helper_build_figure(data_results_df=concatenated_ripple_df, histogram_bins=histogram_bins, earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end, enabled_time_bin_sizes=None,
#                                     main_plot_mode='separate_row_per_session', title=ripple_title)

# concatenated_ripple_df = deepcopy(concatenated_ripple_df)
# concatenated_ripple_df = deepcopy(all_sessions_ripple_df)
# concatenated_ripple_df = deepcopy(all_sessions_ripple_time_bin_df)
concatenated_ripple_df = deepcopy(all_sessions_all_scores_ripple_df)
# concatenated_ripple_df = deepcopy(single_time_bin_size_all_sessions_ripple_df)
concatenated_ripple_df
# ['longest_sequence_length_ratio_diff', 'direction_change_bin_ratio_diff', 'congruent_dir_bins_ratio_diff', 'total_congruent_direction_change_diff']


# variable_name = 'P_Long'
variable_name = 'P_Short' # Shows expected effect - short-only replay prior to delta and then split replays post-delta
# variable_name = 'long_best_pf_peak_x_pearsonr'
# variable_name = 'long_best_jump'
# variable_name = 'wcorr_abs_diff'
# variable_name = 'pearsonr_abs_diff'
# variable_name = 'direction_change_bin_ratio_diff'
# variable_name = 'longest_sequence_length_ratio_diff'
# variable_name = 'long_best_longest_sequence_length_ratio'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# variable_name = 'congruent_dir_bins_ratio_diff'
# variable_name = 'total_congruent_direction_change_diff'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# variable_name = 'long_best_direction_change_bin_ratio'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# 'color':'is_user_annotated_epoch'
# 'color': 'is_user_annotated_epoch', 

px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'color':"is_user_annotated_epoch", 'title': f"'{variable_name}'"} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}

px_scatter_kwargs.pop('color')

# hist_kwargs = dict(color="time_bin_size")
hist_kwargs = dict(color="is_user_annotated_epoch") # , histnorm='probability density'
hist_kwargs.pop('color')
# new_fig_ripples, new_fig_ripples_context = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
#                         px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
#                         time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), legend_title_text='Is User Selected', is_dark_mode=is_dark_mode)


new_fig_ripples, new_fig_ripples_context = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), legend_title_text=None, is_dark_mode=is_dark_mode)

                        

new_fig_ripples = new_fig_ripples.update_layout(fig_size_kwargs)

_extras_output_dict = {}
_extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=0.5, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')

# # Update layout to add a title to the legend
# new_fig_ripples.update_layout(
#     legend_title_text='Is User Selected'  # Add a title to the legend
# )

# fig_to_clipboard(new_fig_ripples, **fig_size_kwargs)
new_fig_ripples_context = new_fig_ripples_context.adding_context_if_missing(num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)
figure_out_paths = save_plotly(a_fig=new_fig_ripples, a_fig_context=new_fig_ripples_context)
new_fig_ripples

In [ ]:
# variable_name = 'short_best_direction_change_bin_ratio'
# variable_name = 'long_best_congruent_dir_bins_ratio'
variable_name = 'short_best_wcorr'
# 'color':'is_user_annotated_epoch'
# 'color': 'is_user_annotated_epoch', 
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'color':"pre_post_delta_category", 'title': f"'{variable_name}'"} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], , 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}
# hist_kwargs = dict(color="time_bin_size")
hist_kwargs = dict(color="pre_post_delta_category") # , histnorm='probability density'
new_fig_ripples, figure_context = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), is_dark_mode=is_dark_mode)
_extras_output_dict = {}
_extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=0.0, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples.show()
figure_out_paths = save_plotly(a_fig=new_fig_ripples, a_fig_context=figure_context)


In [ ]:
# variable_name = 'total_congruent_direction_change_diff'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# variable_name = 'long_best_total_congruent_direction_change'
variable_name = 'wcorr_diff'
# variable_name = 'long_best_wcorr'
# 'color':'is_user_annotated_epoch'
# 'color': 'is_user_annotated_epoch', 
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'color':"is_user_annotated_epoch", 'title': f"'{variable_name}'", 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], 
# hist_kwargs = dict(color="time_bin_size")
hist_kwargs = dict(color="is_user_annotated_epoch") # , histnorm='probability density'
new_fig_ripples, figure_context = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), is_dark_mode=is_dark_mode)
_extras_output_dict = {}
_extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=0.0, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples.show()

figure_context = figure_context.adding_context_if_missing(num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)
figure_out_paths = save_plotly(a_fig=new_fig_ripples, a_fig_context=figure_context)


## Stats Tests

In [ ]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _perform_stats_tests


# stats_variable_name: str = 'P_Short'
# stats_variable_name: str = 'short_best_direction_change_bin_ratio'
# stats_variable_name: str = 'short_best_wcorr'
shuffle_results, p_value, f_value, (dof1, dof2), (variance1, variance2) = _perform_stats_tests(valid_ripple_df, stats_variable_name='short_best_wcorr')


In [ ]:
# Show that wcorr in both periods is higher than shuffles


In [ ]:
## User non-selected:
scatter_title = f'user_approved_ripple_df Several Sessions {variable_name}'
# variable_name = 'wcorr_abs_diff'
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size'}} # , 'color': 'time_bin_size'
new_fig_ripples, figure_context = plotly_pre_post_delta_scatter(data_results_df=deepcopy(user_approved_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), is_dark_mode=is_dark_mode)
_extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=0.5, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples

In [ ]:
# IDEA: The ones with clear replays (diagonal sequences in the decoded posteriors) are by definiition ambiguous, because there's not much difference between the long/short decoders.


In [ ]:
## User non-selected:
scatter_title = f'Non-selected Several Sessions {variable_name}'
# variable_name = 'wcorr_abs_diff'
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size'}} # , 'color': 'time_bin_size'
new_fig_ripples, figure_context = plotly_pre_post_delta_scatter(data_results_df=deepcopy(user_rejected_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end))
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples


In [ ]:
# Laps test
concatenated_ripple_df = deepcopy(all_sessions_simple_pearson_laps_df)

scatter_title = 'Several Sessions'
variable_name = 'wcorr_abs_diff'
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size'}} 
new_fig_ripples, figure_context = plotly_pre_post_delta_scatter(data_results_df=deepcopy(concatenated_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end))
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples


# Plotting functions

In [ ]:
from pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig import PlottingHelpers
# from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import plot_across_sessions_scatter_results

# Example usage:
all_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_df, concatenated_ripple_df=all_sessions_ripple_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   laps_title_prefix=f"Laps", ripple_title_prefix=f"Ripples", save_figures=True, figure_save_extension=['.html','.png'], is_dark_mode=is_dark_mode)
fig_laps, fig_ripples = all_session_figures[0]
# fig_laps.update_layout(fig_size_kwargs)
# fig_ripples.update_layout(fig_size_kwargs)

# fig_laps.show()
fig_ripples.show()
# fig_laps.write_html(f"../output/{TODAY_DAY_DATE}_AcrossSession_fig_laps.html")
# fig_ripples.write_html(f"../output/{TODAY_DAY_DATE}_AcrossSession_fig_ripples.html")


In [ ]:
figure_context = figure_context.adding_context_if_missing(num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)



In [ ]:
fig_to_clipboard(fig_ripples)

In [ ]:
## time_bin version:
all_time_bin_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_time_bin_df, concatenated_ripple_df=all_sessions_ripple_time_bin_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   main_plot_mode='separate_row_per_session',
														   laps_title_prefix=f"Laps_per_time_bin", ripple_title_prefix=f"Ripples_per_time_bin", save_figures=True, figure_save_extension=['.html','.png'], is_dark_mode=is_dark_mode)
fig_time_bin_laps, fig_time_bin_ripples = all_time_bin_session_figures[0]
# fig_time_bin_laps.show()
fig_time_bin_ripples.show()

In [ ]:
## Test collapsed histograms-only results:
histograms_only_all_time_bin_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_time_bin_df, concatenated_ripple_df=all_sessions_ripple_time_bin_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
															# enabled_time_bin_sizes=[0.03, 0.058, 0.10], # [0.03 , 0.044, 0.058, 0.072, 0.086, 0.1]
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   main_plot_mode='default',
														   laps_title_prefix=f"Laps_per_time_bin", ripple_title_prefix=f"Ripples_per_time_bin", save_figures=False, figure_save_extension=['.html','.png'])
histograms_only_fig_time_bin_laps, histograms_only_fig_time_bin_ripples = histograms_only_all_time_bin_session_figures[0]
# histograms_only_fig_time_bin_laps.show()
histograms_only_fig_time_bin_ripples.show()

In [ ]:
import plotly.subplots as sp
import plotly.graph_objs as go



all_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=single_time_bin_size_all_sessions_laps_df, concatenated_ripple_df=single_time_bin_size_all_sessions_ripple_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   laps_title_prefix=f"Laps", ripple_title_prefix=f"Ripples", save_figures=True, figure_save_extension=['.html','.png'], is_dark_mode=is_dark_mode, main_plot_mode='default')
fig_laps, fig_ripples = all_session_figures[0]


In [ ]:
from pyphoplacecellanalysis.Pho2D.plotly.Extensions.plotly_helpers import plotly_pre_post_delta_scatter

# data_results_df: pd.DataFrame = deepcopy(all_sessions_laps_df) # all_sessions_laps_time_bin_df
data_results_df: pd.DataFrame = deepcopy(single_time_bin_size_all_sessions_laps_time_bin_df).infer_objects()

# histogram_bins = 'auto'
histogram_bins: int = 25
new_laps_fig, new_laps_fig_context = plotly_pre_post_delta_scatter(data_results_df=data_results_df, out_scatter_fig=fig_laps, histogram_bins=histogram_bins, px_scatter_kwargs=dict(title='Laps'))


# new_laps_fig = new_laps_fig.update_layout(fig_size_kwargs, 
#     xaxis_title="X Axis Title",
#     yaxis_title="Y Axis Title",
#     legend_title="Legend Title",
#     font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="RebeccaPurple"
#     ),
# )
# Update x-axis labels
# new_laps_fig.update_xaxes(title_text="Num Time Bins", row=1, col=1)
# new_laps_fig.update_xaxes(title_text="Delta-aligned Event Time (seconds)", row=1, col=2)
# new_laps_fig.update_xaxes(title_text="Num Time Bins", row=1, col=3)


_extras_output_dict = {}
_extras_output_dict["y_mid_line"] = new_laps_fig.add_hline(y=0.5, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')

new_laps_fig

# # Update layout to add a title to the legend
# new_fig_ripples.update_layout(
#     legend_title_text='Is User Selected'  # Add a title to the legend
# )

# fig_to_clipboard(new_fig_ripples, **fig_size_kwargs)

new_laps_fig_context: IdentifyingContext = new_laps_fig_context.adding_context_if_missing(epoch='laps', num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)
figure_out_paths = save_plotly(a_fig=new_laps_fig, a_fig_context=new_laps_fig_context)
new_laps_fig

In [ ]:
data_results_df


In [ ]:
np.unique(data_results_df['time_bin_size'])


In [ ]:

data_results_df[data_results_df['time_bin_size'] == 0.005]

# Matplotlib-based versions:

In [ ]:
from pyphoplacecellanalysis.Pho2D.statistics_plotting_helpers import plot_histograms_across_sessions, plot_stacked_histograms

matplotlib.use('Qt5Agg')


In [ ]:

# #TODO 2024-06-05 09:15: - [ ] Define save_matplotlib function
new_laps_fig_context: IdentifyingContext = new_laps_fig_context.adding_context_if_missing(epoch='laps', num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)

In [ ]:
# from Spike3D.PendingNotebookCode import plot_stacked_histograms

# You can use it like this:'long_best_dir_simple_pearsonr'
_out0: "MatplotlibRenderPlots" = plot_histograms_across_sessions(data_type='Laps', session_spec='All Sessions', data_results_df=all_sessions_laps_time_bin_df, time_bin_duration_str="75 ms")
_out1: "MatplotlibRenderPlots" = plot_histograms_across_sessions(data_type='Ripples', session_spec='All Sessions', data_results_df=all_sessions_ripple_time_bin_df, time_bin_duration_str="75 ms")

In [ ]:


fig_to_clipboard(_out0.figures[0])

In [ ]:
## all_sessions_new_laps_df, all_sessions_new_ripple_df
_out0: "MatplotlibRenderPlots" = plot_histograms_across_sessions(data_type='New Laps', session_spec='All Sessions', data_results_df=all_sessions_new_laps_df, time_bin_duration_str="25 ms")
_out1: "MatplotlibRenderPlots" = plot_histograms_across_sessions(data_type='New Ripples', session_spec='All Sessions', data_results_df=all_sessions_new_ripple_df, time_bin_duration_str="25 ms")



In [ ]:
all_sessions_new_laps_df

In [ ]:
len(all_sessions_new_laps_df['session_name'].unique()) # 10 sessions



In [ ]:

matplotlib.use('Qt5Agg')

column_name: str = 'P_Short'

# You can use it like this:
num_unique_sessions: int = all_sessions_laps_time_bin_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
num_unique_time_bins: int = all_sessions_laps_time_bin_df.time_bin_size.nunique(dropna=True)
_laps_histogram_out = plot_stacked_histograms(all_sessions_laps_time_bin_df, 'Laps', f'{num_unique_sessions} Sessions', f"{num_unique_time_bins} tbin sizes", column_name=column_name)

fig_to_clipboard(_laps_histogram_out.figures[0])

num_unique_sessions: int = all_sessions_ripple_time_bin_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
num_unique_time_bins: int = all_sessions_ripple_time_bin_df.time_bin_size.nunique(dropna=True)
_ripple_histogram_out = plot_stacked_histograms(all_sessions_ripple_time_bin_df, 'Ripples', f'{num_unique_sessions} Sessions', f"{num_unique_time_bins} tbin sizes", column_name=column_name)



In [ ]:
all_sessions_ripple_time_bin_df['time_bin_size'].unique()

In [ ]:
# active_time_bin_size: float = 0.072 # 0.058
# active_time_bin_size: float = 0.086
# active_time_bin_size: float = 0.1 # looks the most bimodal with as little of an intermediate value as possible.
active_time_bin_size: float = 0.13444444 # looks the most bimodal with as little of an intermediate value as possible.
_single_time_bin_ripple_histogram_out = plot_stacked_histograms(all_sessions_ripple_time_bin_df[np.isclose(all_sessions_ripple_time_bin_df['time_bin_size'], active_time_bin_size)], 'Ripples', f'{num_unique_sessions} Sessions', f"tbin: {active_time_bin_size:0.3f}s", column_name=column_name)
fig_to_clipboard(_single_time_bin_ripple_histogram_out.figures[0], bbox_inches='tight')


In [ ]:
fig_to_clipboard(_ripple_histogram_out.figures[0])

In [ ]:
_laps_histogram_out.figures[0]

figures_folder

figures_folder

# 2024-03-28 - AcrossSessionTable (PhoDibaPaper2023 formats) .h5 and figure exports

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsVisualizations
from pyphocorehelpers.DataStructure.RenderPlots.MatplotLibRenderPlots import MatplotlibRenderPlots


neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.build_all_known_tables(included_session_contexts, included_h5_paths, should_restore_native_column_types=True)
num_sessions: int = len(long_short_fr_indicies_analysis_table['session_uid'].unique())
print(f'num_sessions: {num_sessions}')

long_short_fr_indicies_analysis_table


def _save_matplotlib_fig(matplotlib_output_container: MatplotlibRenderPlots):

    a_fig_context = matplotlib_output_container.context
    assert len(matplotlib_output_container.saved_figures) == 1
    a_saved_fig_path = matplotlib_output_container.saved_figures[0][0]
    assert a_saved_fig_path.exists()

    if neptuner_run is not None:
        a_full_figure_path_key: str = a_fig_context.get_description(separator='/', include_property_names=True, key_value_separator=':') # .replace(' ', '_')
        # a_full_figure_path_key: str = a_fig_context.get_description(separator=':', include_property_names=True, key_value_separator='|')
        print(f'a_full_figure_path_key: "{a_full_figure_path_key}"')
        # neptuner_run['outputs']['figures'][f"{a_full_figure_path_key}"].upload(a_fig)
        neptuner_run['outputs']['figures'][f"{a_full_figure_path_key}"].upload(a_saved_fig_path.as_posix())
        # neptuner.figures[f"{a_full_figure_path_key}"].upload(a_fig)
        


In [ ]:
# matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
from pyphocorehelpers.DataStructure.RenderPlots.MatplotLibRenderPlots import MatplotlibRenderPlots

matplotlib.use('Qt5Agg')
matplotlib_output_container: MatplotlibRenderPlots = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=num_sessions)

In [ ]:
# graphics_output_dict.figures[0]
# graphics_output_dict.context
_save_matplotlib_fig(matplotlib_output_container)

In [ ]:
graphics_output_dict.saved_figures[0][0] #.plot_data['saved_figures']

In [ ]:
copy_image_to_clipboard(graphics_output_dict['figures'][0])

In [ ]:
neptuner.stop()

## 2024-07-08 - Single Session Plotting


In [ ]:
csv_path = Path(r"K:\scratch\collected_outputs\2024-07-05-kdiba_gor01_two_2006-6-07_16-40-19__withNewKamranExportedReplays-(ripple_marginals_df).csv").resolve()
assert csv_path.exists()

ripple_df = pd.read_csv(csv_path)
ripple_df


In [ ]:
# from Spike3D.PendingNotebookCode import plot_stacked_histograms

# You can use it like this:'long_best_dir_simple_pearsonr'
_out1: "MatplotlibRenderPlots" = plot_histograms_across_sessions(data_type='Ripples', session_spec='All Sessions', data_results_df=ripple_df, time_bin_duration_str="20 ms")

In [ ]:
from pyphoplacecellanalysis.Pho2D.plotly.Extensions.plotly_helpers import plotly_pre_post_delta_scatter

# data_results_df: pd.DataFrame = deepcopy(all_sessions_laps_df) # all_sessions_laps_time_bin_df
# data_results_df: pd.DataFrame = deepcopy(ripple_df[['P_LR', 'P_RL', 'P_Long', 'P_Short', 'ripple_start_t', 'delta_aligned_start_t', 'time_bin_size']]).infer_objects()
data_results_df: pd.DataFrame = deepcopy(ripple_df[['P_Short','delta_aligned_start_t', 'time_bin_size']]).infer_objects()
data_results_df

In [ ]:
print(list(data_results_df.columns)) # ['Unnamed: 0', 'P_LR', 'P_RL', 'P_Long', 'P_Short', 'ripple_idx', 'ripple_start_t', 'session_name', 'delta_aligned_start_t', 'time_bin_size']
['P_LR', 'P_RL', 'P_Long', 'P_Short', 'ripple_start_t', 'delta_aligned_start_t', 'time_bin_size']


In [ ]:
# histogram_bins = 'auto'
histogram_bins: int = 25
new_ripple_fig, new_ripple_fig_context = plotly_pre_post_delta_scatter(data_results_df=data_results_df, out_scatter_fig=None, histogram_bins=histogram_bins, px_scatter_kwargs=dict(title='Ripple'), histogram_variable_name='P_Short')
new_ripple_fig


# new_laps_fig = new_laps_fig.update_layout(fig_size_kwargs, 
#     xaxis_title="X Axis Title",
#     yaxis_title="Y Axis Title",
#     legend_title="Legend Title",
#     font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="RebeccaPurple"
#     ),
# )
# Update x-axis labels
# new_laps_fig.update_xaxes(title_text="Num Time Bins", row=1, col=1)
# new_laps_fig.update_xaxes(title_text="Delta-aligned Event Time (seconds)", row=1, col=2)
# new_laps_fig.update_xaxes(title_text="Num Time Bins", row=1, col=3)


_extras_output_dict = {}
_extras_output_dict["y_mid_line"] = new_ripple_fig.add_hline(y=0.5, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')

new_ripple_fig



# # Update layout to add a title to the legend
# new_fig_ripples.update_layout(
#     legend_title_text='Is User Selected'  # Add a title to the legend
# )

# fig_to_clipboard(new_fig_ripples, **fig_size_kwargs)

# new_laps_fig_context: IdentifyingContext = new_laps_fig_context.adding_context_if_missing(epoch='withNewKamranExportedReplays', num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)
# figure_out_paths = save_plotly(a_fig=new_laps_fig, a_fig_context=new_laps_fig_context)
# new_laps_fig